In [1]:
# Importing libraries #
import re, string
import nltk
import numpy as np
import pandas as pd
import lightgbm as lgb
# import seaborn as sn
import language_tool_python
# import matplotlib.pyplot as plt
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from textacy import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline

tool = language_tool_python.LanguageTool('es-US')
stemmer = SnowballStemmer('spanish')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dmurcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Deifining key functions #

# Preprocessing paragraphs #

def process_paragraph(paragraph):
    """
    This function removes non representative string characters
    input: paragraph extracted from plain text file
    output: paragraph withoput stopwords and punctuation
    """

    # Function for data cleaning #
    def text_cleaning(text_to_clean):
        """
        This function cleans the text data
         """
        text_to_clean = preprocessing.remove.punctuation(text_to_clean)
        text_to_clean = preprocessing.remove.accents(text_to_clean)
        text_to_clean = preprocessing.normalize.whitespace(text_to_clean)
        text_to_clean = preprocessing.normalize.unicode(text_to_clean)
        return text_to_clean
    
    # Function for removing stopwords #
    def stopword(string):
        a = [i for i in string.split() if i not in stopwords.words('spanish')]
        return ' '.join(a)

    # Stemming the paragraph #
    def stemming(text_to_process):
        return ' '.join([ stemmer.stem(i) for i in text_to_process.split() ])

    # Correcting the input pragraph #
    new_paragraph =  stemming(stopword(text_cleaning(tool.correct(paragraph))))

    return new_paragraph

# Testing of functions #
test_paragraph = 'imienlo de dichas areas y la financiacion de los csquemas de pago por servicios ambientales _ lo anterior significa que el hecho que el predio que se restituye el reposo} cumo lo informo corpouraba, se localiza en la ronda hidrica del rio lcon. adcmas que el area que estc ocupa esta establecido como limite dc los humcdales del rio leon y quc segun el pot del municipio dc chigorodo define el area que ocupa este terreno como de preservacion y prolcccion amb'
print(process_paragraph(test_paragraph))

imitenl dich are financiacion esquem pag servici ambiental anterior signif hech predi restitu repos inform corporal localiz rond hidric rio adem are ocup establec limit dc humedal rio leon segun municipi dc vigor defin are ocup terren preservacion produccion acb


In [3]:
raw_text = {"list_victimario":["o años.para la epoca en la que se adquirio el señalando fundo, lasituacion de orden publico en la zona era grave pues hacia presencia laguerrilla del eln, frente juan fernando porras comandado porallas juan carlos, quienes manipulaban a juan joserodriguez diaz, uno de los hijos de los solicitantes; incluso, erallevado de la escuela a entrenamiento sin el consentimiento de su padrey amenazaban con llevarselo para hacer parte de sus filas.el solicitan","eja.manuel maria rodriguez florian conocio a josemercedes calderon y a su conyuge blanca cecilia carreñocardenas porque el comandante juan carlos del eln, le dijo queesa era la persona con quien deberia realizar el negocio de la venta delpredio.al final, el solicitante perdio el vinculo juridico de propiedadque sostenia con el predio a traves del acto administrativo representadoen la resolucion n* 00507 de 28 de enero de 2002, por medio de la cualel","unciarsefrente a los hechos de la solicitud de restitucion de tierras, indicandobasicamente que el citado comandante juan carlos del grupoguerrillero eln al cual se atribuye la autoria del desplazamiento forzadoalegado por los solicitantes como motivo de la dejacion de su heredad,no opero en la zona de ubicacion del predio materia del proceso deacuerdo con el informe n* 9-53815 emitido por la fiscalia general de lanacion. asimismo refirio que el doc","ia, la solvencia del deudor yde sus codeudores, de sus activos y patrimonio, asi como elcomportamiento de pagos; igualmente manifesto haber realizado elnegocio juridico con la persona que era propietaria del inmueble paraese momento?*.una vez evacuadas las pruebas decretadas, el juzgadosegundo civil del circuito especializado en restitucion de tierras decucuta, dispuso remitir el presente asunto a esta colegiatura para lo desu competencia?.del trami","l cual, entre otros aspectos, se relieva quedurante la primera mitad de la decada del 70 y hasta la primera mitad dela decada de los años noventa, el eln es el unico grupo guerrillero condominio en norte de santander. tambien la informacion remitida por elcentro nacional de memorial historica, da cuenta de sucesosconcernientes con desapariciones forzadas, asesinatos selectivos,masacres y secuestros perpetrados por el eln, en la zona de cucuta,tanto "," de memorial historica, da cuenta de sucesosconcernientes con desapariciones forzadas, asesinatos selectivos,masacres y secuestros perpetrados por el eln, en la zona de cucuta,tanto en la zona urbana como rural (incluyendo el corregimiento deaguaclara), acaecidos entre los años de 1995 y 20012 como por iguallo dice el documento diagnostico departamental de norte desantander?. incluso, de la misma zona de aguaclara, la javilla y puerto19 fis. 75 a 98","la misma zona de aguaclara, la javilla y puerto19 fis. 75 a 98 cano. 1 principal.20 fls. 415 a 436 cdno. principal tres (3).21 (...) la presencia del eln ha sido preponderante por largo tiempo y hasta finales de los noventa sobrelas otras organizaciones alzadas en armas que tienen presencia en el departamento. los frentes deleln ubicados en norte de santander hacen parte del frente de guerra nororiental, la estructura belicacon presencia historica e","or largo tiempo y hasta finales de los noventa sobrelas otras organizaciones alzadas en armas que tienen presencia en el departamento. los frentes deleln ubicados en norte de santander hacen parte del frente de guerra nororiental, la estructura belicacon presencia historica en el departamento y la mas activa del eln, cuya expansion se ha dado alrededorde la explotacion del oleoducto caño limon-coveñas. esta organizacion tiene presencia a traves de l","n norte de santander hacen parte del frente de guerra nororiental, la estructura belicacon presencia historica en el departamento y la mas activa del eln, cuya expansion se ha dado alrededorde la explotacion del oleoducto caño limon-coveñas. esta organizacion tiene presencia a traves de losfrentes efrain pabon pabon y juan fernando porras en el centro y sur del departamento, el carlosarmando cacua guerrero en la zona del catatumbo; y el frente carlo","os testigos citados dieron a conocerque en la zona de ubicacion del predio han estado presentes gruposarmados al margen de la ley, particularmente el eln. sobre esteparticular, alvaro julio jaimes suarez, habitante de aguaclara,manifesto: (...) pues grupos al margen de la ley pues seria mentira decir queno hay, pero que asi si que esten de base en el alla no (...)'2. tambien lodijo luis francisco pabon arias, persona que llego junto con lossolicitan","n predio en la mismaparcelacion y quien llego para el mismo tiempo en que lo hizo elreclamante, narro: (...) si claro, por ahi estaba como que era el eln cuandoesa epoca (...)'28. finalmente, aldemar zuleta giraldo, quientambien entro a la parcelacion santa monica en el año 1991, conto: (...)pues es que eso no es ningun secreto, esa zona ha sido todo el tiempo zonaroja, para esa epoca estaban los elenos (...)?.circunstancias todas que reflejan el as","o por elreclamante que se vio obligado a salir de la zona para evitar que sumenor hijo juan jose fuere reclutado por alias juan carlos,comandante del eln y al intentar vender el predio, debio hacerlo solo a28 para la corte la expresion 'con ocasion del conflicto armado, inserta en la definicion operativa devictima' establecida en el articulo 3? de la ley 1448 de 2011, delimita el universo de victimasbeneficiarias de la ley de manera constitucional y","nte, en declaracion rendida ante el juzgadopreciso que la salida de su heredad devino por (...) el problema que tuvecon la guerrilla, la guerrilla de eln. los hechos los cuales fue que a mi hijojuan jose rodriguez, que para entonces tenia 12 años, se lo querianllevar: teniamos miedo y decidimos vender la mejora para irnos de ahi, salirde ahi30,por su parte, la tambien solicitante flor merinda diazlerma manifesto en el juzgado que: (...) un dia yo es","no podiamos demorarlo tanto (...)'1, aseverando luegoque quien le hizo esas manifestaciones fue justamente el guerrillerojuan carlos perteneciente al eln, de quien asumia integraba esegrupo (...) pues donde daba tantas ordenes, a donde le decia, a donde medijo que se llevaba a mi hijo, ellos eran los de ese poder; entonces, pues yome imagino, entonces pues que paso? el le dijo que el ahi iba a quedar, ledijo a mi marido que ahi iba a quedar el que e","cadade los años 70 a la actualidad?', se limita a enunciar -de manera genericapor demas- la presencia de diversos actores armados legales (epl,farc y eln, autodefensas y bacrim). sin embargo, cuando se habladel sector en el que se ubica el predio de que aqui se trata (corregimientode aguaclara), amen de alguna intervencion de grupos de autodefensasy respecto de actos ocurridos en el segundo semestre de 1999 y de alli34 corte suprema de justicia. sal","a hicimos nosotros (...)'38 y de afirmar que alli tuvo sus hijos estudiando,cuando derechamente fue preguntada acerca de reclutamientos quehiciere el eln en ese escuela, manifesto de manera rotunda que (...)no, mis hijos pues estudiaron ahi, siempre estaba pendiente de mis hijos (...)37y que a pesar de que igual sabia que el hijo del solicitante manuelmaria (...) si estudiaba (...)'8 en el mismo establecimiento, jamas suponi directamente (...) ni po","ndo y trabajando en la parcelatambien cuando no tenian estudio; igual que todos nosotros (...)*?, en puntode su conocimiento sobre intimidaciones del eln para forzar alreclutamiento de los niños que alli se encontraban, manifesto que (...)no doctor, nunca se, digo yo que nunca se presento eso porque cuandollegaban a reclutar, recogen y se llevan sin mediar mucho; me voy a llevar asu hijo y listo se lo llevaban*,en similares terminos se pronuncio su ","uela en ese lugar y que el hijodel solicitante (...) estudio en esa escuela (...)Ñ0, cuanto concernio con elacusado reclutamiento por disposicion del eln, de manera enfaticarespondio que de eso no supo (...) ni por comentarios; no, ni porcomentarios, ni mucho menos que lo haiga vivido yo (...)'91 y aun menos seentero que ello hubiere ocurrido con el hijo del reclamante*? o con otrosvecinos suyos*Ñ.como se ve, ni uno solo de esos testigos hace por lo","mente eso (...)8,de lo que si hablo fue de jose calderon de quien, amende precisar que no le constaba que tuviere vinculo alguno con esaguerrilla del eln*, dejo en claro que en momento alguno le presionopara realizar el negocio indicando que (...) amenazas no hubieron y no conel señor que me compro la parcela que me dio el millon trescientos; elproblema era con el señor juan carlos que era el que comandaba losguerrilleros que andaban por ahi en el e","carlos (...)?* sin mas informacion queesa, ninguno de sus otros vecinos que aqui rindieron declaracion supode algun comandante o siquiera miembro del eln que con ese nombreo alias, tuviere presencia en la zona para ese entonces. los informes decontexto de violencia arrimados al plenario ni siquiera lo citan.tampoco hay prueba de que constituya patron deconducta muy propio del eln que operaba por entonces en esa zona,ese de obligar a parceleros a que","los informes decontexto de violencia arrimados al plenario ni siquiera lo citan.tampoco hay prueba de que constituya patron deconducta muy propio del eln que operaba por entonces en esa zona,ese de obligar a parceleros a que vendieran sus tierras a favor de quiendispusiera el grupo guerrillero? hasta el mismo solicitante reconocioque (...) hasta cuando yo me vine, no habian otras personas que tenian quevender; no se que pasaria tiempo despues??.esta","nuelmaria rodriguez florian ni haber participado en los actos deviolencia que ocasionaron el aludido despojo ni muchos menos haberpertenecido a algun grupo armado al margen de la ley; de otro lado,resalto que dio cumplimiento a los requisitos exigidos para laadjudicacion de la heredad y efectuo el pago ante el extinto incora,adquiriendolo en consecuencia de manera legal. finalmente indico quede acuerdo con registro unico de victimas, se tiene conocimiento de","e la decadade los años 70 a la actualidad?', se limita a enunciar -de manera genericapor demas- la presencia de diversos actores armados legales (epl,farc y eln, autodefensas y bacrim). sin embargo, cuando se habladel sector en el que se ubica el predio de que aqui se trata (corregimientode aguaclara), amen de alguna intervencion de grupos de autodefensasy respecto de actos ocurridos en el segundo semestre de 1999 y de alli34 corte suprema de justici","ca cecilia carreñocardenas porque el comandante juan carlos del eln, le dijo queesa era la persona con quien deberia realizar el negocio de la venta delpredio.al final, el solicitante perdio el vinculo juridico de propiedadque sostenia con el predio a traves del acto administrativo representadoen la resolucion n* 00507 de 28 de enero de 2002, por medio de la cualel incora declaro la caducidad administrativa de la resolucion n* 0390de 7 de abril de 1","ndamental a la restitucion?.el ministerio publico no hizo pronunciamiento alguno.se considera:debese comenzar diciendo que la naturaleza y filosofia delproceso de restitucion de tierras que contempla la ley 1448 de 2011, yaha venido decantandose con suficiencia por lo que no viene al caso caeren repeticiones innecesarias. apenas si importa memorar quebasicamente presupone la conjuncion de una serie de supuestos que, almargen de la inscripcion del bi","cion del bien para asiestablecer si fue consecuencia de suceso relacionado con el conflictoarmado.para propositos tales, debe comenzarse diciendo que elplenario ofrece con suficiencia las probanzas que dan efectiva cuentaque en la zona en la que se situa la requerida heredad, y por las mismasepocas en que se afirma que sobrevino el abandono y despojo delpredio, mediaron sucesos de orden publico que por su gravedad y porlos actores involucrados, sin ","n efectiva cuentaque en la zona en la que se situa la requerida heredad, y por las mismasepocas en que se afirma que sobrevino el abandono y despojo delpredio, mediaron sucesos de orden publico que por su gravedad y porlos actores involucrados, sin hesitacion pueden asimilarse como propiosdel conflicto armado.asi se comprueba, por ejemplo, con la informacioncontenida y recolectada en el documento de analisis del contexto deviolencia del municipio de"," a lasfalencias probatorias que se dejaron expuestas, de inmediato quiebranesa ensayada teoria.en efecto: se dijo en la solicitud que justo a partir delprobable reclutamiento de su hijo, los solicitantes optaron por vender elbien, diciendo a ese respecto que (...) teniamos miedo y decidimos venderla mejora para irnos de ahi, salir de ahi (...)92 razon por la cual (...) eso yolo negocie con un señor, eso hace tanto tiempo, el señor lo vi una sola vez","presionopara realizar el negocio indicando que (...) amenazas no hubieron y no conel señor que me compro la parcela que me dio el millon trescientos; elproblema era con el señor juan carlos que era el que comandaba losguerrilleros que andaban por ahi en el entonces (...)*8. este pasaje, amenque por si solo dicientemente descarta cualquier perfida intencion delseñalado comprador de sacar provecho de la situacion, habria queanalizarlo con ese otro, di","ño de 2002, por hechos sucedidos en elmunicipio antioqueño de yondo (casabe) y comunicados enbarrancabermeja, diciendose que (...) cada rato se ve en elpueblo que la gente dice que viene la guerrilla que vienenlos paracos y le toca a uno salir corriendo entoncesdebido a eso decidi venirme (...)'9. desde luego que no podriaacusarse que esa omision de denuncia encontrare justificacion en eltemor de sufrir represalias. pues ello no constituyo obstaculo"," la actualidad?', se limita a enunciar -de manera genericapor demas- la presencia de diversos actores armados legales (epl,farc y eln, autodefensas y bacrim). sin embargo, cuando se habladel sector en el que se ubica el predio de que aqui se trata (corregimientode aguaclara), amen de alguna intervencion de grupos de autodefensasy respecto de actos ocurridos en el segundo semestre de 1999 y de alli34 corte suprema de justicia. sala de casacion penal. au","te durantecinco años.para la epoca en la que se adquirio el señalando fundo, lasituacion de orden publico en la zona era grave pues hacia presencia laguerrilla del eln, frente juan fernando porras comandado porallas juan carlos, quienes manipulaban a juan joserodriguez diaz, uno de los hijos de los solicitantes; incluso, erallevado de la escuela a entrenamiento sin el consentimiento de su padrey amenazaban con llevarselo para hacer parte de sus filas.el s","e la escuela a entrenamiento sin el consentimiento de su padrey amenazaban con llevarselo para hacer parte de sus filas.el solicitante rechazo que la guerrilla  vincularaforzadamente a su hijo, quien para la epoca tenia doce años de edad,pues a su esposa le manifestaron que a la semana siguiente se lollevaban, circunstancia por la cual, temiendo ser asesinado por el grupoarmado, decidio desplazarse de la region en el año de 1996, abandonarforzosamente el ","ste dijo dentro del marco de laactuacion administrativa ante la unidad de restitucion de tierras: (...)el problema radico basicamente, en que como la guerrilla nos pedia que lecolaboraramos, yo nunca me preste para eso, entonces ellos me miraban mal,por oponerme, luego de eso mi hijo juan jose rodriguez diaz, en eseentonces tenia 12 años, y la guerrilla queria reclutarlo, y yo no deje que se lollevaran, lo encerre en la casa, no lo deje ir mas a la escuel","nca me preste para eso, entonces ellos me miraban mal,por oponerme, luego de eso mi hijo juan jose rodriguez diaz, en eseentonces tenia 12 años, y la guerrilla queria reclutarlo, y yo no deje que se lollevaran, lo encerre en la casa, no lo deje ir mas a la escuela, para no darlesoportunidad de pronto que se lo llevaran, con estos hechos yo sabia que ellosme iban a asesinan por eso tome la decision de irme junto con mi familia de lazona (...) fue mas o men","ando luegoque se sintio forzado a abandonar el predio (...) por oponerme a todo loque querian ellos, sobre todo por no dejar llevar a mi hijo para la guerrilla,bueno y tambien que cuando se enterraron que me iba, me obligaron a realizarel negocio con la persona que ellos indicaban (sic).posteriormente, en declaracion rendida ante el juzgadopreciso que la salida de su heredad devino por (...) el problema que tuvecon la guerrilla, la guerrilla de eln. los h","indicaban (sic).posteriormente, en declaracion rendida ante el juzgadopreciso que la salida de su heredad devino por (...) el problema que tuvecon la guerrilla, la guerrilla de eln. los hechos los cuales fue que a mi hijojuan jose rodriguez, que para entonces tenia 12 años, se lo querianllevar: teniamos miedo y decidimos vender la mejora para irnos de ahi, salirde ahi30,por su parte, la tambien solicitante flor merinda diazlerma manifesto en el juzgado qu",").posteriormente, en declaracion rendida ante el juzgadopreciso que la salida de su heredad devino por (...) el problema que tuvecon la guerrilla, la guerrilla de eln. los hechos los cuales fue que a mi hijojuan jose rodriguez, que para entonces tenia 12 años, se lo querianllevar: teniamos miedo y decidimos vender la mejora para irnos de ahi, salirde ahi30,por su parte, la tambien solicitante flor merinda diazlerma manifesto en el juzgado que: (...) un di","echos victimizantes aqui relatadosdatan de 1996), ni por asomo hacen mencion de situacion alguna relativacon reclutamiento forzado propiciado por las guerrillas de izquierda.lo que tampoco se descubre de otras pruebas; ni siquiera apartir de las manifestaciones de quienes fueron residentes del sectorpara la misma epoca, ninguno de los cuales acuso que hubiere pasadopor situaciones semejantes que implicaren ese eventual reclutamientode sus hijos o que hubi","ra reversarel negocio a cuyo proposito explico aquel que pasados mas o menos(...) unos seis meses (...)'2 desde la venta, el comprador escogido por laguerrilla (jose calderon) (...) me quedo a dar cuatrocientos mil pesosmas; pero cuando yo mande a la señora a buscarlo, el le dijo que, que no memandaba la plata sino que volviera yo para que le devolviera la plata y que elme iba a devolver la parcela (...)'*Ñ diciendo luego que (...) el no me mandonada. el ","tio y practicamente eso (...)8,de lo que si hablo fue de jose calderon de quien, amende precisar que no le constaba que tuviere vinculo alguno con esaguerrilla del eln*, dejo en claro que en momento alguno le presionopara realizar el negocio indicando que (...) amenazas no hubieron y no conel señor que me compro la parcela que me dio el millon trescientos; elproblema era con el señor juan carlos que era el que comandaba losguerrilleros que andaban por ahi","en ese sentido se trae a cuento, por una parte, que conforme con los estudios realizados por el centronacional de memoria historica, el actuar de las guerrillas de izquierda en materia de desplazamientogeneralmente apuntaba (...) no al despojo de propiedades, como si a la construccion de corredorestransitorios, bien para el desplazamiento de tropas o de insumos para la guerra e incluso hasta el traficode base de coca (...) (htip://www.centrodememoriahisto","nicipio antioqueño de yondo (casabe) y comunicados enbarrancabermeja, diciendose que (...) cada rato se ve en elpueblo que la gente dice que viene la guerrilla que vienenlos paracos y le toca a uno salir corriendo entoncesdebido a eso decidi venirme (...)'9. desde luego que no podriaacusarse que esa omision de denuncia encontrare justificacion en eltemor de sufrir represalias. pues ello no constituyo obstaculo parahacerlo frente al otro caso como que se a","a esa epoca estaban los elenos (...)?.circunstancias todas que reflejan el asedio constante ypermanente de grupos guerrilleros y posteriormente de lasautodefensas, al punto mismo que, por el despliegue que les fue dadoen su momento, calificarian de entrada como notorios, todo lo cual,daria pie para entender que de veras se trato de una zona mediada porla constante presencia de grupos al margen de la ley.22 ver: https:/avww.laopinion.com.co/judicial/con-20-va","e los años 70 a la actualidad?', se limita a enunciar -de manera genericapor demas- la presencia de diversos actores armados legales (epl,farc y eln, autodefensas y bacrim). sin embargo, cuando se habladel sector en el que se ubica el predio de que aqui se trata (corregimientode aguaclara), amen de alguna intervencion de grupos de autodefensasy respecto de actos ocurridos en el segundo semestre de 1999 y de alli34 corte suprema de justicia. sala de casacion ","go, cuando se habladel sector en el que se ubica el predio de que aqui se trata (corregimientode aguaclara), amen de alguna intervencion de grupos de autodefensasy respecto de actos ocurridos en el segundo semestre de 1999 y de alli34 corte suprema de justicia. sala de casacion penal. auto de 22 de abril de 2015. ap2005-2015radicado n* 45361. magistrado ponente: dr. gustavo enrique malo fernandez (incidente derestitucion de predios y cancelacion de titulos o"],"list_perdida":["a declaro la caducidad administrativa de la resolucion n* 0390de 7 de abril de 1993, por la que otrora se le habia adjudicado el predio,por razon del abandono del mismo y por el incumplimiento de lasobligaciones adquiridas.luego de la comentada declaratoria de caducidadadministrativa, que por demas, nunca le fue informada o notificadapersonalmente al solicitante, el incora, a traves de la resolucion n485 de 30 de julio de 2002 adjudico nuevamente el inmu","e santander.cuanto refiere con la relacion juridica de los solicitantesrespecto del fundo que aqui se pide restituir y para la epoca del despojoo del abandono, basta con decir que de acuerdo con la informacioninscrita en el folio de matricula inmobiliaria n* 260-151578**, el dichobien fue en comienzo adjudicado por el incora a la solicitante medianteresolucion n* 00390 de 7 abril de 199375 (anotacion n* 1); dominio quetuvo hasta cuando la misma entidad e"," por la que se adjudico el fundo a la ahora propietariablanca cecilia carreño cardenas.y como se anuncio que los hechos victimizantes quemotivaron el abandono y el aducido despojo de la parcela sucedieronen 1996, a la que le subsiguio la señalada resolucion de caducidadproferida el 28 de enero de 2002, tampoco puede ofrecer duda que secompasa con el supuesto factico-temporal previsto en el articulo 75 dela ley.asimismo, la condicion de victima de los rec","las probanzas que dan efectiva cuentaque en la zona en la que se situa la requerida heredad, y por las mismasepocas en que se afirma que sobrevino el abandono y despojo delpredio, mediaron sucesos de orden publico que por su gravedad y porlos actores involucrados, sin hesitacion pueden asimilarse como propiosdel conflicto armado.asi se comprueba, por ejemplo, con la informacioncontenida y recolectada en el documento de analisis del contexto deviolencia d","ra definir casos similares, es aspecto queen cualquier supuesto apenas si envuelve la gran probabilidad, enmucho muy alta eso si, de desplazamientos, abandonos y despojos debienes en el señalado sector por disimiles factores asociados a eseconflicto; es a eso a lo que refieren varios de los indicios y presuncionesque se gobiernan en la ley 1448 y que ciertamente aprovechan alreclamante para darle fuerza a sus pedimentos. pero por muy juiciosasque sean la","gar ala apurada y bien desventurada tesis de que toda traslacion o dejacionde bienes en zona afectada por el conflicto armado implica per sedespojo o abandono forzado o desplazamiento. ni mas faltaba quepudieren generalizarse todos los supuestos con tan simplista solucion.en buenas cuentas: que el aqui solicitante apenas iria amitad de camino en tanto que en estas contiendas, no basta la palmariacomprobacion de esa calidad de victima como tampoco con acr","sa condicion de victima no entraña per se eldespojo ni se presume como indefectible causa de la enajenacionposterior como tampoco el desplazamiento o abandono de un bienimplica por si solo un aprovechamiento del ulterior adquirente niconstituye necesariamente la razon de su venta ni mucho menos, bajoese solo antecedente, convierte cualquier traspaso del derecho endespojo.justo por ello, es menester que a la par de ese contextoviolento rondante (o incluso","el eligiera, no el que eldispusiera*?.incumbe ahora relievar que en estos asuntos la prueba delos hechos victimizantes y su relacion con el despojo o abandono, sesatisface -por lo menos en comienzo- a partir de las propiasmanifestaciones de quienes fungen como victimas. en efecto: eseecumenico principio probatorio de que nadie puede en juicio hacerse supropia prueba, aplicable por regla general a toda controversia judicial,encuentra aqui singular excepci","do la junta directivadel incora (...)'85, aclarase de una vez que nunca obedecio, cual sedijere inopinadamente en la solicitud, *(...) con motivo del abandono delbien (...) (hecho octavo)$Ñ, asunto ese que ni por asomo se mencionaen el señalado acto.con esa precision, remembrase que en la resolucion deadjudicacion de 1993 -proferida en vigencia de la ley 135 de 1961-, seindico que el adjudicatario no podia enajenar el predio sino pasadosquince años (num."," en el municipio de yondo -antioquia-declarado el 2 de abril de 2002 en el municipio de barrancabermeja. deotro lado, estimo no ser viable considerar despojo la decision contendidaen un acto administrativo por medio del cual el otrora incora declarola caducidad administrativa de la resolucion de adjudicacion hecha a losreclamantes mediante resolucion n* 0390 de 7 de abril de 1993, pueseste se presume legal. indico tambien que de acuerdo con lo consignad","a buena fe exenta de culpa, alego no conocer a manuelmaria rodriguez florian ni haber participado en los actos deviolencia que ocasionaron el aludido despojo ni muchos menos haberpertenecido a algun grupo armado al margen de la ley; de otro lado,resalto que dio cumplimiento a los requisitos exigidos para laadjudicacion de la heredad y efectuo el pago ante el extinto incora,adquiriendolo en consecuencia de manera legal. finalmente indico quede acuerdo co","o, condicion igualmente reconocida consu inclusion en el registro unico de victimas por hechos violentosdeclarados el 2 de abril de 2002. en torno al despojo alegado estimo$ fls. 7 a 8 cdno. del tribunal.7 fl 56. ib. o8 fls. 59 a 63 ib.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 7haberse sucedido por cuanto la enajenacion del predio por parte delsolicitante se llevo a cabo dentro","ucuta,norte de santander.cuanto refiere con la relacion juridica de los solicitantesrespecto del fundo que aqui se pide restituir y para la epoca del despojoo del abandono, basta con decir que de acuerdo con la informacioninscrita en el folio de matricula inmobiliaria n* 260-151578**, el dichobien fue en comienzo adjudicado por el incora a la solicitante medianteresolucion n* 00390 de 7 abril de 199375 (anotacion n* 1); dominio quetuvo hasta cuando la m","o el fundo a la ahora propietariablanca cecilia carreño cardenas.y como se anuncio que los hechos victimizantes quemotivaron el abandono y el aducido despojo de la parcela sucedieronen 1996, a la que le subsiguio la señalada resolucion de caducidadproferida el 28 de enero de 2002, tampoco puede ofrecer duda que secompasa con el supuesto factico-temporal previsto en el articulo 75 dela ley.asimismo, la condicion de victima de los reclamantes noofrece rep","as que dan efectiva cuentaque en la zona en la que se situa la requerida heredad, y por las mismasepocas en que se afirma que sobrevino el abandono y despojo delpredio, mediaron sucesos de orden publico que por su gravedad y porlos actores involucrados, sin hesitacion pueden asimilarse como propiosdel conflicto armado.asi se comprueba, por ejemplo, con la informacioncontenida y recolectada en el documento de analisis del contexto deviolencia del municip","suasion de que la perdida delderecho sobre el predio ocurrio por la intercesion del conflicto armado olo que es lo mismo, que de veras se trato de un despojo en lascondiciones que refiere el articulo 74 de la ley 1448 de 2011.en efecto: incumbe previamente señalar, por las razonesque luego se diran, que aunque en una zona y en una epocadeterminadas, aparezca claramente establecido un grave contexto deviolencia correspondiente con el conflicto armado, lo","asos similares, es aspecto queen cualquier supuesto apenas si envuelve la gran probabilidad, enmucho muy alta eso si, de desplazamientos, abandonos y despojos debienes en el señalado sector por disimiles factores asociados a eseconflicto; es a eso a lo que refieren varios de los indicios y presuncionesque se gobiernan en la ley 1448 y que ciertamente aprovechan alreclamante para darle fuerza a sus pedimentos. pero por muy juiciosasque sean las pruebas s","ena de llegar ala apurada y bien desventurada tesis de que toda traslacion o dejacionde bienes en zona afectada por el conflicto armado implica per sedespojo o abandono forzado o desplazamiento. ni mas faltaba quepudieren generalizarse todos los supuestos con tan simplista solucion.en buenas cuentas: que el aqui solicitante apenas iria amitad de camino en tanto que en estas contiendas, no basta la palmariacomprobacion de esa calidad de victima como tamp"," si esto es consecuencia deaquello. casi que sobra decir, por supuesto, que el plenoconvencimiento sobre esa condicion de victima no entraña per se eldespojo ni se presume como indefectible causa de la enajenacionposterior como tampoco el desplazamiento o abandono de un bienimplica por si solo un aprovechamiento del ulterior adquirente niconstituye necesariamente la razon de su venta ni mucho menos, bajoese solo antecedente, convierte cualquier traspaso","rior adquirente niconstituye necesariamente la razon de su venta ni mucho menos, bajoese solo antecedente, convierte cualquier traspaso del derecho endespojo.justo por ello, es menester que a la par de ese contextoviolento rondante (o incluso sin el) se enseñe en todo caso prueba enconcreto por cuya entidad se concluya que de veras si ocurrio un hechotocante con el conflicto armado interno que, a su vez, determino la ventadel bien.con apoyo en esas prev"," armado interno que, a su vez, determino la ventadel bien.con apoyo en esas previas precisiones, incumbe entoncesaplicarse a calificar si ese acusado despojo, con las aristas expuestaspor el solicitante, fue de veras propiciado o condicionado por algunsupuesto que se equiparase con hechos que quepa involucrar dentro delamplio espectro de conflicto armado interno2,pues bien: en el asunto de que aqui se trata, se adujo por elreclamante que se vio obligado","ar el que el eligiera, no el que eldispusiera*?.incumbe ahora relievar que en estos asuntos la prueba delos hechos victimizantes y su relacion con el despojo o abandono, sesatisface -por lo menos en comienzo- a partir de las propiasmanifestaciones de quienes fungen como victimas. en efecto: eseecumenico principio probatorio de que nadie puede en juicio hacerse supropia prueba, aplicable por regla general a toda controversia judicial,encuentra aqui singu","es que enseñan que las cosasno sucedieron precisamente del modo en que las narraron lossolicitantes, a lo menos no en cuanto hace con las razones del despojo.tal sucede, primeramente, al parar mientes en lo que señalael documento de analisis de contexto del area rural de cucuta que fuerarecaudado durante la etapa administrativa en cuya semblanza historica,que recoge un espacio de tiempo que abarca incluso desde la decadade los años 70 a la actualidad?',","mportaba a manuel maria eravender el predio, sea quien fuere. lo que de suyo deja muy en vilo queen condiciones semejantes se hubiere configurado ese despojo que segobierna en el articulo 74 de la ley 1448 de 2011.pero la extrañeza de la situacion no para ahi. sigueperfilandose cuando del mismo modo se repara en lo que asevero elsolicitante sobre ese posterior encuentro que casualmente tuvo con elreferido guerrillero. como que en ese sentido explico que","os por el centronacional de memoria historica, el actuar de las guerrillas de izquierda en materia de desplazamientogeneralmente apuntaba (...) no al despojo de propiedades, como si a la construccion de corredorestransitorios, bien para el desplazamiento de tropas o de insumos para la guerra e incluso hasta el traficode base de coca (...) (htip://www.centrodememoriahistorica.gov.co/descargastinformes2010fierra_conflicto/despojo_tierras_baja.paf. p. 71.)","mos para la guerra e incluso hasta el traficode base de coca (...) (htip://www.centrodememoriahistorica.gov.co/descargastinformes2010fierra_conflicto/despojo_tierras_baja.paf. p. 71.) y por otra, que esos datos reseñados en el documento de analisis de contexto(dac) area rural de cucuta -punto 2.1.1. desplazamiento forzado y despojo de tierras por partede este accionar insurgente- (fis. 80 a 81 vto. cdno. 1 principal), en el que se hizo mencion de otrosd","p. 71.) y por otra, que esos datos reseñados en el documento de analisis de contexto(dac) area rural de cucuta -punto 2.1.1. desplazamiento forzado y despojo de tierras por partede este accionar insurgente- (fis. 80 a 81 vto. cdno. 1 principal), en el que se hizo mencion de otrosdos sucesos que son los que mas se acomodan a ese invocado patron y que se corresponden con lasdeclaraciones de reclamantes de tierras consignadas en los formulanos distinguidos","itud de restitucion de tierras, indicandobasicamente que el citado comandante juan carlos del grupoguerrillero eln al cual se atribuye la autoria del desplazamiento forzadoalegado por los solicitantes como motivo de la dejacion de su heredad,no opero en la zona de ubicacion del predio materia del proceso deacuerdo con el informe n* 9-53815 emitido por la fiscalia general de lanacion. asimismo refirio que el documento aportado como prueba dedesplazamiento sufri","del proceso deacuerdo con el informe n* 9-53815 emitido por la fiscalia general de lanacion. asimismo refirio que el documento aportado como prueba dedesplazamiento sufrido por los reclamantes da cuenta de su ocurrenciaen el mes de marzo de 2002 en el municipio de yondo -antioquia-declarado el 2 de abril de 2002 en el municipio de barrancabermeja. deotro lado, estimo no ser viable considerar despojo la decision contendidaen un acto administrativo por medio del","empo transcurridoentre la venta de las mejoras y la fecha en que se declaro la caducidadadministrativa -seis años- no informaron al incora el supuestodesplazamiento forzado sufrido. resalto que desde el año 1996,y fls. 179 a 184 cdno. 1 principal.2 fl. 187 lb.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 5blanca cecilia carreño cardenas se encuentra en el prediocomo poseedora y por ello c","imiento dehechos victimizantes contra la reclamante ocurridos en el municipio deyondo -antioquia- el 25 de marzo de 2002 sin que se hubieredenunciado desplazamiento anterior en el corregimiento de aguaclaradel municipio de cucuta.por su parte, la unidad de restitucion de tierras, enrepresentacion de los solicitantes, adujo que estaba acreditada lacalidad de victima de manuel maria rodriguez florian asicomo el elemento de temporalidad, por cuanto se vio obligad","manuel mariarodriguez florian y flor merinda diaz lerma con la heredadobjeto de la solicitud de restitucion como asi mismo su condicion devictimas de desplazamiento, cuanto incumbe ahora es detenerse en elanalisis de la situacion que provoco la dejacion del bien para asiestablecer si fue consecuencia de suceso relacionado con el conflictoarmado.para propositos tales, debe comenzarse diciendo que elplenario ofrece con suficiencia las probanzas que dan efectiva ","le orientacion para definir casos similares, es aspecto queen cualquier supuesto apenas si envuelve la gran probabilidad, enmucho muy alta eso si, de desplazamientos, abandonos y despojos debienes en el señalado sector por disimiles factores asociados a eseconflicto; es a eso a lo que refieren varios de los indicios y presuncionesque se gobiernan en la ley 1448 y que ciertamente aprovechan alreclamante para darle fuerza a sus pedimentos. pero por muy juiciosas","ien desventurada tesis de que toda traslacion o dejacionde bienes en zona afectada por el conflicto armado implica per sedespojo o abandono forzado o desplazamiento. ni mas faltaba quepudieren generalizarse todos los supuestos con tan simplista solucion.en buenas cuentas: que el aqui solicitante apenas iria amitad de camino en tanto que en estas contiendas, no basta la palmariacomprobacion de esa calidad de victima como tampoco con acreditar5400131210022015003","ncimiento sobre esa condicion de victima no entraña per se eldespojo ni se presume como indefectible causa de la enajenacionposterior como tampoco el desplazamiento o abandono de un bienimplica por si solo un aprovechamiento del ulterior adquirente niconstituye necesariamente la razon de su venta ni mucho menos, bajoese solo antecedente, convierte cualquier traspaso del derecho endespojo.justo por ello, es menester que a la par de ese contextoviolento rondante","na parte, que conforme con los estudios realizados por el centronacional de memoria historica, el actuar de las guerrillas de izquierda en materia de desplazamientogeneralmente apuntaba (...) no al despojo de propiedades, como si a la construccion de corredorestransitorios, bien para el desplazamiento de tropas o de insumos para la guerra e incluso hasta el traficode base de coca (...) (htip://www.centrodememoriahistorica.gov.co/descargastinformes2010fierra_co"," materia de desplazamientogeneralmente apuntaba (...) no al despojo de propiedades, como si a la construccion de corredorestransitorios, bien para el desplazamiento de tropas o de insumos para la guerra e incluso hasta el traficode base de coca (...) (htip://www.centrodememoriahistorica.gov.co/descargastinformes2010fierra_conflicto/despojo_tierras_baja.paf. p. 71.) y por otra, que esos datos reseñados en el documento de analisis de contexto(dac) area rural de ","espojo_tierras_baja.paf. p. 71.) y por otra, que esos datos reseñados en el documento de analisis de contexto(dac) area rural de cucuta -punto 2.1.1. desplazamiento forzado y despojo de tierras por partede este accionar insurgente- (fis. 80 a 81 vto. cdno. 1 principal), en el que se hizo mencion de otrosdos sucesos que son los que mas se acomodan a ese invocado patron y que se corresponden con lasdeclaraciones de reclamantes de tierras consignadas en los formu","25que en cualquier caso, desde la fecha de la notificacion de laadjudicacion al aqui solicitante (abril de 1993) y hasta cuando dijo quese produjo su desplazamiento y ya luego la venta del bien (a mediadosde 1996), habian transcurrido ya mas de los tres años para principiar apagar el capital. y sin embargo no lo hizo; con todo y que tambien enotro aparte habia afirmado que en el dicho inmueble habia permanecido(...) como alrededor de unos cuatro años y medio, ","n barrancabermeja, era delicado (...)'??.tampoco se vislumbra porque, teniendo a mano la oportunidad de referiren ese momento y alli, sobre su previo desplazamiento de cucuta, nohicieron, sin embargo, asi fuere una minima mencion de lo otroraacontecido. en fin: ni entonces ni despues.la conjuncion de todas estas circunstancias no muestra conclaridad el derecho que dijeron tener los solicitantes para obtener larestitucion. pues que aqui ello no quedo demostrado","cion de tierras, indicandobasicamente que el citado comandante juan carlos del grupoguerrillero eln al cual se atribuye la autoria del desplazamiento forzadoalegado por los solicitantes como motivo de la dejacion de su heredad,no opero en la zona de ubicacion del predio materia del proceso deacuerdo con el informe n* 9-53815 emitido por la fiscalia general de lanacion. asimismo refirio que el documento aportado como prueba dedesplazamiento sufrido por l","doentre la venta de las mejoras y la fecha en que se declaro la caducidadadministrativa -seis años- no informaron al incora el supuestodesplazamiento forzado sufrido. resalto que desde el año 1996,y fls. 179 a 184 cdno. 1 principal.2 fl. 187 lb.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 5blanca cecilia carreño cardenas se encuentra en el prediocomo poseedora y por ello cumplio c","purada y bien desventurada tesis de que toda traslacion o dejacionde bienes en zona afectada por el conflicto armado implica per sedespojo o abandono forzado o desplazamiento. ni mas faltaba quepudieren generalizarse todos los supuestos con tan simplista solucion.en buenas cuentas: que el aqui solicitante apenas iria amitad de camino en tanto que en estas contiendas, no basta la palmariacomprobacion de esa calidad de victima como tampoco con acreditar54","n por eso tome la decision de irme junto con mi familia de lazona (...) fue mas o menos para mitad de año en 1996 (...) precisando luegoque se sintio forzado a abandonar el predio (...) por oponerme a todo loque querian ellos, sobre todo por no dejar llevar a mi hijo para la guerrilla,bueno y tambien que cuando se enterraron que me iba, me obligaron a realizarel negocio con la persona que ellos indicaban (sic).posteriormente, en declaracion rendida ante","delante (memorese que los hechos victimizantes aqui relatadosdatan de 1996), ni por asomo hacen mencion de situacion alguna relativacon reclutamiento forzado propiciado por las guerrillas de izquierda.lo que tampoco se descubre de otras pruebas; ni siquiera apartir de las manifestaciones de quienes fueron residentes del sectorpara la misma epoca, ninguno de los cuales acuso que hubiere pasadopor situaciones semejantes que implicaren ese eventual recluta",") si estudiaba (...)'8 en el mismo establecimiento, jamas suponi directamente (...) ni por comentarios de la comunidad (...)'92 que aquelhubiere sido forzado a realizar entrenamientos por cuenta de dichogrupo para que hiciere parte de sus filas*. otro tanto mencionoaldemar zuleta giraldo, quien amen de reafirmar que en lavereda funcionaba la escuela*? e incluso que tambien sabia que el hijodel aqui solicitante se la pasaba (...) estudiando y trabajando ","baja.paf. p. 71.) y por otra, que esos datos reseñados en el documento de analisis de contexto(dac) area rural de cucuta -punto 2.1.1. desplazamiento forzado y despojo de tierras por partede este accionar insurgente- (fis. 80 a 81 vto. cdno. 1 principal), en el que se hizo mencion de otrosdos sucesos que son los que mas se acomodan a ese invocado patron y que se corresponden con lasdeclaraciones de reclamantes de tierras consignadas en los formulanos di"," proteccion por la que aquise propende. pues no cabe perder de mira que en estos escenarios, esmenester ademas llegar a la clara persuasion de que la perdida delderecho sobre el predio ocurrio por la intercesion del conflicto armado olo que es lo mismo, que de veras se trato de un despojo en lascondiciones que refiere el articulo 74 de la ley 1448 de 2011.en efecto: incumbe previamente señalar, por las razonesque luego se diran, que aunque en una zona y"],"list_solicitante":["guerrilla del eln, frente juan fernando porras comandado porallas juan carlos, quienes manipulaban a juan joserodriguez diaz, uno de los hijos de los solicitantes; incluso, erallevado de la escuela a entrenamiento sin el consentimiento de su padrey amenazaban con llevarselo para hacer parte de sus filas.el solicitante rechazo que la guerrilla  vincularaforzadamente a su hijo, quien para la epoca tenia doce años de edad,pues a su esposa le manifestaron que a","ntes; incluso, erallevado de la escuela a entrenamiento sin el consentimiento de su padrey amenazaban con llevarselo para hacer parte de sus filas.el solicitante rechazo que la guerrilla  vincularaforzadamente a su hijo, quien para la epoca tenia doce años de edad,pues a su esposa le manifestaron que a la semana siguiente se lollevaban, circunstancia por la cual, temiendo ser asesinado por el grupoarmado, decidio desplazarse de la region en el año de 1996, ","e, como a las diez, llego este a la casa y le entrego aaquel la suma de $1.300.000.00 diciendo que era lo unico que tenia asicomo tambien le quito al solicitante los titulos entregados por el incoradiciendosele del mismo modo que ya estaba listo para irse por lo quedebio entonces salir al otro dia con su familia a valledupar en dondevivieron dos meses y despues se trasladaron para barrancabermeja.manuel maria rodriguez florian conocio a josemercedes caldero","enas porque el comandante juan carlos del eln, le dijo queesa era la persona con quien deberia realizar el negocio de la venta delpredio.al final, el solicitante perdio el vinculo juridico de propiedadque sostenia con el predio a traves del acto administrativo representadoen la resolucion n* 00507 de 28 de enero de 2002, por medio de la cualel incora declaro la caducidad administrativa de la resolucion n* 0390de 7 de abril de 1993, por la que otrora se le h","gaciones adquiridas.luego de la comentada declaratoria de caducidadadministrativa, que por demas, nunca le fue informada o notificadapersonalmente al solicitante, el incora, a traves de la resolucion n485 de 30 de julio de 2002 adjudico nuevamente el inmueble, en estaocasion, a favor de blanca cecilia carreño cardenas, quienactualmente y segun consta en el folio de matricula inmobiliaria n* 260-151578, ostenta su titularidad.del tramite ante el juzgado:5400","ndobasicamente que el citado comandante juan carlos del grupoguerrillero eln al cual se atribuye la autoria del desplazamiento forzadoalegado por los solicitantes como motivo de la dejacion de su heredad,no opero en la zona de ubicacion del predio materia del proceso deacuerdo con el informe n* 9-53815 emitido por la fiscalia general de lanacion. asimismo refirio que el documento aportado como prueba dedesplazamiento sufrido por los reclamantes da cuenta de"," de 7 de abril de 1993, pueseste se presume legal. indico tambien que de acuerdo con lo consignadoen la resolucion n* 007 de 18 de enero de 2002, los solicitantes fueronemplazados y se les designo curador, y durante el tiempo transcurridoentre la venta de las mejoras y la fecha en que se declaro la caducidadadministrativa -seis años- no informaron al incora el supuestodesplazamiento forzado sufrido. resalto que desde el año 1996,y fls. 179 a 184 cdno. 1 pri","cuenta estas de la existencia de amenazas dereclutamiento de menores, a pesar de tener hijos estudiando en la mismaescuela donde lo hacia el hijo del solicitante. de otro lado, y en punto ala invocada buena fe exenta de culpa, alego no conocer a manuelmaria rodriguez florian ni haber participado en los actos deviolencia que ocasionaron el aludido despojo ni muchos menos haberpertenecido a algun grupo armado al margen de la ley; de otro lado,resalto que dio ","azamiento anterior en el corregimiento de aguaclaradel municipio de cucuta.por su parte, la unidad de restitucion de tierras, enrepresentacion de los solicitantes, adujo que estaba acreditada lacalidad de victima de manuel maria rodriguez florian asicomo el elemento de temporalidad, por cuanto se vio obligado aabandonar su predio en el año 1996 por temor a ser asesinado por elgrupo guerrillero que lo amenazo, condicion igualmente reconocida consu inclusion "," restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 7haberse sucedido por cuanto la enajenacion del predio por parte delsolicitante se llevo a cabo dentro de un contexto de violencia. consideroasi que deberia concederse la implorada proteccion del derechofundamental a la restitucion?.el ministerio publico no hizo pronunciamiento alguno.se considera:debese comenzar diciendo que la naturaleza y filosofia delproceso de restitucion","ca, ubicado en lavereda la javilla, corregimiento de aguaclara, municipio de cucuta,norte de santander.cuanto refiere con la relacion juridica de los solicitantesrespecto del fundo que aqui se pide restituir y para la epoca del despojoo del abandono, basta con decir que de acuerdo con la informacioninscrita en el folio de matricula inmobiliaria n* 260-151578**, el dichobien fue en comienzo adjudicado por el incora a la solicitante medianteresolucion n* 0039"," acuerdo con la informacioninscrita en el folio de matricula inmobiliaria n* 260-151578**, el dichobien fue en comienzo adjudicado por el incora a la solicitante medianteresolucion n* 00390 de 7 abril de 199375 (anotacion n* 1); dominio quetuvo hasta cuando la misma entidad emitio la resolucion n* 007 de 28de enero de 2002 por la que decreto la caducidad administrativa de laadjudicacion?f y que aparece inscrita en la anotacion n* 3 del mismofolio. ya luego "," otra. 10villamizar, se tienen informes periodisticos que dan cuenta de lasituacion de orden publico que abarcan ese periodo de tiempo narradopor los solicitantes??.igualmente, varios de los testigos citados dieron a conocerque en la zona de ubicacion del predio han estado presentes gruposarmados al margen de la ley, particularmente el eln. sobre esteparticular, alvaro julio jaimes suarez, habitante de aguaclara,manifesto: (...) pues grupos al margen de la ","a decir queno hay, pero que asi si que esten de base en el alla no (...)'2. tambien lodijo luis francisco pabon arias, persona que llego junto con lossolicitantes a la misma parcelacion santa monica y a quien le fueasignada tambien en el año 1993 la parcela n* 5, quien en torno de ellomanifesto que: (...) pues mire: yo decirle a usted 'no han habido grupos almargen de la ley, toda la vida lo han existido en la frontera, porque eso si todala vida han habido ","dono forzado o desplazamiento. ni mas faltaba quepudieren generalizarse todos los supuestos con tan simplista solucion.en buenas cuentas: que el aqui solicitante apenas iria amitad de camino en tanto que en estas contiendas, no basta la palmariacomprobacion de esa calidad de victima como tampoco con acreditar540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 12diamantinamente sucesos de vio","la ventadel bien.con apoyo en esas previas precisiones, incumbe entoncesaplicarse a calificar si ese acusado despojo, con las aristas expuestaspor el solicitante, fue de veras propiciado o condicionado por algunsupuesto que se equiparase con hechos que quepa involucrar dentro delamplio espectro de conflicto armado interno2,pues bien: en el asunto de que aqui se trata, se adujo por elreclamante que se vio obligado a salir de la zona para evitar que sumenor h","ra entonces tenia 12 años, se lo querianllevar: teniamos miedo y decidimos vender la mejora para irnos de ahi, salirde ahi30,por su parte, la tambien solicitante flor merinda diazlerma manifesto en el juzgado que: (...) un dia yo estaba hablando conuna vecina mia, le dicen la negra', pero el nombre de ella es adila, adilafranco, yo estaba hablando con ella diciendole que mi hijo se habia vueltoinsoportable, que no sabia que le pasaba, cuando entonces llego ","o discutido corresponde aun formalismo que reclama (...) simple constatacion de validez. pues contodo y que es verdad que la especial condicion de la solicitante y losfines perseguidos, (...) implican una necesaria flexibilizacion del materialprobatorio (...) jamas ha significado que las atestaciones de los desmovilizadoso el contenido de los medios suasorios arrimados, deba ser asumido comocierto por si mismo, o pueda dejarse de lado su valoracion dentro d"," aqui visos de prosperidaddesde que se advierten fundadas razones que enseñan que las cosasno sucedieron precisamente del modo en que las narraron lossolicitantes, a lo menos no en cuanto hace con las razones del despojo.tal sucede, primeramente, al parar mientes en lo que señalael documento de analisis de contexto del area rural de cucuta que fuerarecaudado durante la etapa administrativa en cuya semblanza historica,que recoge un espacio de tiempo que abar","era rotunda que (...)no, mis hijos pues estudiaron ahi, siempre estaba pendiente de mis hijos (...)37y que a pesar de que igual sabia que el hijo del solicitante manuelmaria (...) si estudiaba (...)'8 en el mismo establecimiento, jamas suponi directamente (...) ni por comentarios de la comunidad (...)'92 que aquelhubiere sido forzado a realizar entrenamientos por cuenta de dichogrupo para que hiciere parte de sus filas*. otro tanto mencionoaldemar zuleta gi","tanto mencionoaldemar zuleta giraldo, quien amen de reafirmar que en lavereda funcionaba la escuela*? e incluso que tambien sabia que el hijodel aqui solicitante se la pasaba (...) estudiando y trabajando en la parcelatambien cuando no tenian estudio; igual que todos nosotros (...)*?, en puntode su conocimiento sobre intimidaciones del eln para forzar alreclutamiento de los niños que alli se encontraban, manifesto que (...)no doctor, nunca se, digo yo que n","hijo y listo se lo llevaban*,en similares terminos se pronuncio su tambien vecino luisfrancisco pabon arias, quien luego de afirmar que, en efecto, elsolicitante manuel maria (...) tenia un hijo menor (estudiando en la38 fl 5264. cdno. principal tres (3). record: 00.11.28.37 lb. record: 00.12.12.Ñ lb. record: 00.12.37.39 lb. record: 00.12.58.40 b. record: 00.13.04.41 ib. record: 00.19.30.42 [b. record: 00.19.45.43 [b, record: 00.20.20.540013121002201500389 ","mo sentido loexpuso luis gabriel osorio rojas, quien ademas de admitir suconocimiento sobre la existencia de la escuela en ese lugar y que el hijodel solicitante (...) estudio en esa escuela (...)Ñ0, cuanto concernio con elacusado reclutamiento por disposicion del eln, de manera enfaticarespondio que de eso no supo (...) ni por comentarios; no, ni porcomentarios, ni mucho menos que lo haiga vivido yo (...)'91 y aun menos seentero que ello hubiere ocurrido c","reclutamiento, con todoy que advirtieron varios de ellos que sus hijos igual estudiaban en laescuela para esa misma epoca en que lo hacia el hijo del solicitante. es *que ni siquiera supieron de la existencia de ese señalado comandanteen esa zona para entonces.y sin que haya como decir que se trata de declaracionessesgadas en perjuicio del solicitante o a favor del opositor si es que,44 fl. 524. cdno. principal tres (3). record: 00.35.04.45 [b, record: 00.3","de la existencia de ese señalado comandanteen esa zona para entonces.y sin que haya como decir que se trata de declaracionessesgadas en perjuicio del solicitante o a favor del opositor si es que,44 fl. 524. cdno. principal tres (3). record: 00.35.04.45 [b, record: 00.35.12.46 b, record: 00.35.37.47 ib, record: 00.36.29.48 b. record: 00.36.58.49 fb. record: 00.37.01.50 [b. record: 00.53.13.$1 [b. record: 00.53.33.52 lb. record: 00.54.06.53 [b. record: 00.57.","ntencion de ocultar o desfigurar la verdad.a fin de cuentas, lo unico que le da piso a esa teoria fue loque comentaron sobre ese respecto los propios solicitantes, quienes detodos modos debieron admitir sin ambages que (...) ahi no, hasta quenosotros salimos de ahi, no habia habido reclutamiento de nadie (...)**explicando luego que (...) pues yo no se si a los hijos de ellos (de lostestigos) se los llevarian cuando querian y se los traian cuando querian, no","tenian (...)5.como fuere, aun dejando a un lado ese extraño panoramaque acaso tenga suficiente virtud para infirmar la veracidad de lo alegadopor los solicitantes y mas bien dando por descontado, a partir de esaespecial cualidad probatoria que recubre su dicho, que a pesar de todoy contra todo, fue enteramente cierto lo de la amenaza de reclutamientode su hijo por cuenta de ese grupo guerrillero (que al parecer solo a ellosafecto), ni aun en ese supuesto te","toapunta derechamente a la restitucion de tierras desposeidas.y de ello no hay aqui suficiente constancia probatoria. puessin dejar al margen que los solicitantes trataron de abroquelarse en quefueron esos descritos episodios enlazados con el conflicto los quepropiciaron el temor para salir del predio como tambien para venderlo,algunas otras circunstancias que refleja el expediente y anejas a lasfalencias probatorias que se dejaron expuestas, de inmediato q","n expuestas, de inmediato quiebranesa ensayada teoria.en efecto: se dijo en la solicitud que justo a partir delprobable reclutamiento de su hijo, los solicitantes optaron por vender elbien, diciendo a ese respecto que (...) teniamos miedo y decidimos venderla mejora para irnos de ahi, salir de ahi (...)92 razon por la cual (...) eso yolo negocie con un señor, eso hace tanto tiempo, el señor lo vi una sola vez queno se nombre ni nada; se que era de por alla ","ta listo para que se vaya. me quitaron los titulosde la tierra, yo sali y el quedo con su parcela (...)*1.sin embargo, a la par de esas menciones, el solicitanterefirio tambien algunas otras circunstancias que bien vistas no seacomodan precisamente con esas previas situaciones narradas. porejemplo, lo que concierne con ese insolito hecho de que el adquirentedel bien -el mismo que fuera escogido por alias juan carlos sin laautorizacion del solicitante- de un","emplo, lo que concierne con ese insolito hecho de que el adquirentedel bien -el mismo que fuera escogido por alias juan carlos sin laautorizacion del solicitante- de un momento a otro pretendiera reversarel negocio a cuyo proposito explico aquel que pasados mas o menos(...) unos seis meses (...)'2 desde la venta, el comprador escogido por laguerrilla (jose calderon) (...) me quedo a dar cuatrocientos mil pesosmas; pero cuando yo mande a la señora a buscarlo","antecedente suya de quela idea era, como que asi se lo dijo el propio juan carlos, que elcomprador calderon se quedara ahi en el fundo y asi el propiosolicitante lo dijo desde que (...) ya yo entendi que el iba a quedar ahi, quea mi lo que me tocaba era, que la parcelita me la iban a quitar (...)25, al final,sin embargo, nunca explico la razon por la que no consintio en la ofrecidadevolucion del predio; asunto que quizas encuentre explicacion en queya para ","ticulo 74 de la ley 1448 de 2011.pero la extrañeza de la situacion no para ahi. sigueperfilandose cuando del mismo modo se repara en lo que asevero elsolicitante sobre ese posterior encuentro que casualmente tuvo con elreferido guerrillero. como que en ese sentido explico que (...) al señorjuan carlos, despues, al cabo de unos años, no se cuantos años habrianpasado, por ahi unos dos o tres años, me lo encontre en barrancabermeja; mesaludo, me invito a tomar","dio para tomar la decision de vender el predio) sinoincluso, quien les obligo a ceder el terreno y no precisamente a favor delcomprador con el que el solicitante venia haciendo tratos sino al queaquel dispuso. aun menos si se memora que ese mismo guerrillero,66 (b. record: 00.26.29.67 lb. record: 00.22.40.$8 [b. record: 00.32.53.$9 lb. record: 00.27.06.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez flo","raba por entonces en esa zona,ese de obligar a parceleros a que vendieran sus tierras a favor de quiendispusiera el grupo guerrillero? hasta el mismo solicitante reconocioque (...) hasta cuando yo me vine, no habian otras personas que tenian quevender; no se que pasaria tiempo despues??.estado de cosas que no permite sino resumir el asunto enque, en esa vereda y por esa epoca, y respecto de una generalidad deparceleros con situaciones de veras muy semejante","esumir el asunto enque, en esa vereda y por esa epoca, y respecto de una generalidad deparceleros con situaciones de veras muy semejantes a las de lossolicitantes, solo a estos intentaron reclutarle un hijo y, asimismo, no masque ellos fueron obligados a vender el terreno. algo extraño por decir lomenos.70 fl 64 cdno. 1 principal.71 fl 5264. cdno. principal tres (3). record: 00.10.35.72 en ese sentido se trae a cuento, por una parte, que conforme con los es","nteres alguno (...) entanto que f...) en los dos (2) años siguientes se causaran intereses del cuatropor ciento (4Ñ) anual (...).pues bien: cuando el solicitante fue expresamentepreguntado si habia efectuado los pagos a que estaba obligado con elincora, expresamente dijo que (...) no, porque eso a uno le dan tres añosmuertos para que empiece a pagar y lo que si estaba pagando era el seguroque tiene uno que pagar, un seguro por la tierra; eso si lo estaba pa","n que tal aplicabaestrictamente frente al valor de capital; que no respecto de los intereses,por lo menos no de los sucedidos luego de dos años. y el solicitanteno da cuenta que hubiere efectuado el pago de estos, ni siquiera a partirdel segundo año. pero incluso, ni en el mejor de los supuestos, esto es,ni entendiendo acaso que ese seguro que dijo haber pagado secorrespondia mas bien con esos reditos, no puede perderse de mira74 fls, 54 a 56 lb.75 fl. 56 i","rmalizacion de tierras de manuel maria rodriguez florian y otra. 25que en cualquier caso, desde la fecha de la notificacion de laadjudicacion al aqui solicitante (abril de 1993) y hasta cuando dijo quese produjo su desplazamiento y ya luego la venta del bien (a mediadosde 1996), habian transcurrido ya mas de los tres años para principiar apagar el capital. y sin embargo no lo hizo; con todo y que tambien enotro aparte habia afirmado que en el dicho inmueble","egacion concerniente con el bajisimo precio por el que resultocedido el inmueble ($1.300.000.00)Ñ. porque si para ese entonces eladjudicatario y aqui solicitante, tal cual lo acepto, ni siquiera habiaempezado a sufragar el capital correspondiente al valor de la parcela(acaso tampoco esos intereses ya devengados), en sana logica,tampoco podria pretender que por la venta del predio obtuviere un montoequivalente al precio de un bien que tampoco el habia pagado","cio, las obligaciones de pago al incora setrasladaban al posterior compradort0, todo, sin dejar de mencionar queese monto que se dijo recibido por el solicitante, curiosamente seasemeja mucho con el valor de las mejoras que aparece relacionadoen la misma resolucion y que se tasaron en una suma de$1.361.919.00. lo que vale concatenar justo ahora con lo que en sumomento expuso el reclamante en cuanto reconocio con franqueza que(...) yo lo que estaba vendiendo"," tierra porque la tierra practicamente yo no se la habiapagado al incora todavia (....)\"8*.en fin: no habria como decir que esa venta, que a voces delsolicitante fue solo de las mejoras -que no de la tierra - realmente sehizo por un precio muy por debajo de su justo valor. amen que esamisma aseveracion al propio tiempo dejaria muy en vilo la veracidad deesa mencion, tambien suya, de que antes de que alias juan carlos leobligara a ceder el bien a favor de jo",", cual lo refiere conclaridad la resolucion que declaro la caducidad, habia emitido esaprevia autorizacion para la venta, de suyo implica que el aqui solicitanteen tanto adjudicatario inicial, reclamo y solicito el comentado permisopara vender el que por demas, supone gestiones que demandan alguntiempo y preparacion que no se compasan con un escenario signado porla inminencia y necesidad de vender prontamente.hasta quizas fue por ello mismo, esto es, porque","ion que no se compasan con un escenario signado porla inminencia y necesidad de vender prontamente.hasta quizas fue por ello mismo, esto es, porque elsolicitante no se avino con esos (...) tramites necesarios para lalegalizacion de la transferencia del bien a favor del comprador (...), quejose mercedes calderon opto mejor por retractarse del negociocomo lo manifestare en su momento el reclamante.para rematar, llama la atencion que antes de la solicitud dequ","actarse del negociocomo lo manifestare en su momento el reclamante.para rematar, llama la atencion que antes de la solicitud deque aqui se trata, los solicitantes nunca hubieren puesto enconocimiento los graves hechos que dijeron sucedidos en aguaclara quehasta implicaron el riesgo directo de ser asesinados*e* cuando, encontrario, no tuvieron mayor inconveniente en dar cuenta de su1. la prohibicion para el adjudicatario de transferir por acto entre vivos la","idades necesarias para la adjudicacion original conforme el inciso primero (...)88 art. 51 ley 135 de 1961 (mod. art. 20 ley 30 de 1988).89 señalo el solicitante manuel maria que (...) viendo lo que estaba sucediendo, con la oposicionmia, yo hable con mi compañera y le dije a mi me van a matar por lo que estoy haciendo, entonces noscolocamos de acuerdo y decidimos vender el predio para podemos ir, y evitar la muerte mia (...) (fl. 64cdno. 1 principal).54001"," otroraacontecido. en fin: ni entonces ni despues.la conjuncion de todas estas circunstancias no muestra conclaridad el derecho que dijeron tener los solicitantes para obtener larestitucion. pues que aqui ello no quedo demostrado con suficiencia.traduce que en tanto se echa aqui de menos esa necesariacuanto palmaria conexion que debe existir entre el suceso victimizante yla venta, que es presupuesto sine quanon para que tenga exito lapretension restitutoria","or consecuencia, excluyase del registro detierras despojadas y abandonadas forzadamente, la inscripcion queotrora se hiciera a favor de los señalados solicitantes respecto del prediodenominado parcela n* 12 santa monica, ubicado en la vereda lajavilla, del corregimiento aguaclara del municipio de cucuta (norte desantander), al que le corresponde el folio de matricula inmobiliaria n*260-151578 y el numero predial 00-01-0002-0216-00, y que apareceidentificado"," que, almomento de adquirir el predio, habia asumido por el termino de quinceaños.posteriormente, el citado comandante juan carlos, le diola orden al reclamante que negociara el bien con jose mercedescalderon quien era la persona que debia quedarse con el predio; asipues, una noche, como a las diez, llego este a la casa y le entrego aaquel la suma de $1.300.000.00 diciendo que era lo unico que tenia asicomo tambien le quito al solicitante los titulos entre","forme n* 9-53815 emitido por la fiscalia general de lanacion. asimismo refirio que el documento aportado como prueba dedesplazamiento sufrido por los reclamantes da cuenta de su ocurrenciaen el mes de marzo de 2002 en el municipio de yondo -antioquia-declarado el 2 de abril de 2002 en el municipio de barrancabermeja. deotro lado, estimo no ser viable considerar despojo la decision contendidaen un acto administrativo por medio del cual el otrora incora decl","ontendidaen un acto administrativo por medio del cual el otrora incora declarola caducidad administrativa de la resolucion de adjudicacion hecha a losreclamantes mediante resolucion n* 0390 de 7 de abril de 1993, pueseste se presume legal. indico tambien que de acuerdo con lo consignadoen la resolucion n* 007 de 18 de enero de 2002, los solicitantes fueronemplazados y se les designo curador, y durante el tiempo transcurridoentre la venta de las mejoras y l","ara que se formulasen los correspondientes alegatos de conclusion.oportunamente la opositora estimo que fueron factoreseconomicos los que llevaron al reclamante a salir del predio, pues que,de acuerdo con lo vertido en los testimonios recaudados, el fundo noposeia cultivos, servicios, mejoras ni vias para su explotacion yadicionalmente, tampoco pago siquiera una de las cuotas del creditootorgado para su adquisicion el cual obtuvo por sorteo y posterioradju"," consecuencia de manera legal. finalmente indico quede acuerdo con registro unico de victimas, se tiene conocimiento dehechos victimizantes contra la reclamante ocurridos en el municipio deyondo -antioquia- el 25 de marzo de 2002 sin que se hubieredenunciado desplazamiento anterior en el corregimiento de aguaclaradel municipio de cucuta.por su parte, la unidad de restitucion de tierras, enrepresentacion de los solicitantes, adujo que estaba acreditada laca","ampoco puede ofrecer duda que secompasa con el supuesto factico-temporal previsto en el articulo 75 dela ley.asimismo, la condicion de victima de los reclamantes noofrece reparo. desde luego que queda de plano acreditada con su solainclusion en el registro correspondiente? como asimismo a partir de lasdeclaraciones que estos rindieren en la unidad administrativa especialde gestion de restitucion de tierras despojadas y ante el juzgado, poraquello de la bue",")2s, en tanto que zoraidasanchez ramirez, tambien propietaria de un predio en la mismaparcelacion y quien llego para el mismo tiempo en que lo hizo elreclamante, narro: (...) si claro, por ahi estaba como que era el eln cuandoesa epoca (...)'28. finalmente, aldemar zuleta giraldo, quientambien entro a la parcelacion santa monica en el año 1991, conto: (...)pues es que eso no es ningun secreto, esa zona ha sido todo el tiempo zonaroja, para esa epoca estaba","ciados a eseconflicto; es a eso a lo que refieren varios de los indicios y presuncionesque se gobiernan en la ley 1448 y que ciertamente aprovechan alreclamante para darle fuerza a sus pedimentos. pero por muy juiciosasque sean las pruebas sobre ese contexto como diques a tener en cuenta,solamente comportan signos generalizados que no constituyen reglasfijas que apliquen para cualquier evento mas o menos semejante.con lo que viene de decirse no se esta sig","e con hechos que quepa involucrar dentro delamplio espectro de conflicto armado interno2,pues bien: en el asunto de que aqui se trata, se adujo por elreclamante que se vio obligado a salir de la zona para evitar que sumenor hijo juan jose fuere reclutado por alias juan carlos,comandante del eln y al intentar vender el predio, debio hacerlo solo a28 para la corte la expresion 'con ocasion del conflicto armado, inserta en la definicion operativa devictima' e","einta años y el hijo menor que ya tieneveintiseis años; nunca nos llegaron con el cuento de tal cosa (...)** comotampoco supo que al hijo de los aqui reclamantes, le hubiere sucedidoalgo semejante a cuyo proposito explico que (...) pues no le puedo decirsi, porque los hijos mios nunca llegaron a ese punto que decir uno, tenianalgun campamento, pues nunca yo tuve conocimiento de eso (...)* comotampoco supo que se llevaran niños de la escuela a esos campos d","por comentarios; no, ni porcomentarios, ni mucho menos que lo haiga vivido yo (...)'91 y aun menos seentero que ello hubiere ocurrido con el hijo del reclamante*? o con otrosvecinos suyos*Ñ.como se ve, ni uno solo de esos testigos hace por lo menosalguna tangencial referencia a esos peligros de reclutamiento, con todoy que advirtieron varios de ellos que sus hijos igual estudiaban en laescuela para esa misma epoca en que lo hacia el hijo del solicitante. e","descarta cualquier perfida intencion delseñalado comprador de sacar provecho de la situacion, habria queanalizarlo con ese otro, dicho tambien por el reclamante, conforme conel cual, y al final de cuentas, cuanto le importaba a manuel maria eravender el predio, sea quien fuere. lo que de suyo deja muy en vilo queen condiciones semejantes se hubiere configurado ese despojo que segobierna en el articulo 74 de la ley 1448 de 2011.pero la extrañeza de la situa","l), en el que se hizo mencion de otrosdos sucesos que son los que mas se acomodan a ese invocado patron y que se corresponden con lasdeclaraciones de reclamantes de tierras consignadas en los formulanos distinguidos con los numerosid 25853 (fl. 81 lb.) e id 75059 (f. 81 vto. ib.), refieren en su orden a un caso ocurrido en uncorregimiento de cucuta pero distinto y distante de aguaclara y el segundo, a un asunto cuya solicitudjudicial fue resuelta por esta "," relacionadoen la misma resolucion y que se tasaron en una suma de$1.361.919.00. lo que vale concatenar justo ahora con lo que en sumomento expuso el reclamante en cuanto reconocio con franqueza que(...) yo lo que estaba vendiendo era la mejora, porque era lo del trabajo queyo habia hecho; no la tierra porque la tierra practicamente yo no se la habiapagado al incora todavia (....)\"8*.en fin: no habria como decir que esa venta, que a voces delsolicitante fu","nsferencia del bien a favor del comprador (...), quejose mercedes calderon opto mejor por retractarse del negociocomo lo manifestare en su momento el reclamante.para rematar, llama la atencion que antes de la solicitud deque aqui se trata, los solicitantes nunca hubieren puesto enconocimiento los graves hechos que dijeron sucedidos en aguaclara quehasta implicaron el riesgo directo de ser asesinados*e* cuando, encontrario, no tuvieron mayor inconveniente e"],"list_decision":["","veinte de marzo de dos mil dieciocho.radicacion n* 540013121002201500389 01magistrado ponente: nelson ruiz hernandez.ref.: solicitud de restitucion y formalizacion detierras de manuel maria rodriguez florian yflor merinda diaz lerma.discutido y aprobado por la sala en sesion de 21 de febrerode 2018, segun acta n* 05 de la misma fecha.decidese la solicitud de restitucion y formalizacion detierras presentada por manuel maria rodriguez florian y flormerinda ","z lerma.discutido y aprobado por la sala en sesion de 21 de febrerode 2018, segun acta n* 05 de la misma fecha.decidese la solicitud de restitucion y formalizacion detierras presentada por manuel maria rodriguez florian y flormerinda diaz lerma a cuya prosperidad se opone blancacecilia carreño cardenas.antecedentes:en ejercicio de la facultad otorgada por el articulo 82 de laley 1448 de 2011, la unidad administrativa especial de gestion derestitucion de t","iva especial de gestion derestitucion de tierras despojadas, presento solicitud de restitucion y540013121002201500389 01 arsolicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 2formalizacion de tierras a nombre de manuel maria rodriguezflorian y flor merinda diaz lerma, actuando por conducto deprocuradora judicial designada por la unidad administrativaespecial. de gestion de restitucion de tierrasdespojadas -direcc","o solicitud de restitucion y540013121002201500389 01 arsolicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 2formalizacion de tierras a nombre de manuel maria rodriguezflorian y flor merinda diaz lerma, actuando por conducto deprocuradora judicial designada por la unidad administrativaespecial. de gestion de restitucion de tierrasdespojadas -direccion territorial norte de santander-y con fundamento en la ley 1448 d","eccion territorial norte de santander-y con fundamento en la ley 1448 de 2011, solicitaron se les protegieresu derecho fundamental a la restitucion y formalizacion de tierras,respecto del predio rural denominado parcela n? 12, el cual presenta unarea de 15 hectareas y 4618 m?, distinguido con el folio de matriculainmobiliaria n* 260-151578 y numero predial 00-01-0002-0216-00,ubicado en la vereda la javilla del corregimiento aguaclara delmunicipio de cucut"," region en el año de 1996, abandonarforzosamente el predio y, en consecuencia, como unica solucion540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 3enajenarlo. la situacion narrada le impidio pagar el credito que, almomento de adquirir el predio, habia asumido por el termino de quinceaños.posteriormente, el citado comandante juan carlos, le diola orden al reclamante que negociara el bie","folio de matricula inmobiliaria n* 260-151578, ostenta su titularidad.del tramite ante el juzgado:540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 4el juzgado segundo civil del circuito especializado enrestitucion de tierras de cucuta admitio la accion ordenandose lainscripcion y sustraccion provisional del comercio del predio objeto de lamisma, asi como la suspension de los procesos ju","to forzado sufrido. resalto que desde el año 1996,y fls. 179 a 184 cdno. 1 principal.2 fl. 187 lb.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 5blanca cecilia carreño cardenas se encuentra en el prediocomo poseedora y por ello cumplio con los requisitos exigidos por la leypara que se realizara la adjudicacion a su favor mediante resolucion n*0485 de 30 de julio de 2002. manifesto en","de3 fls. 297 a 304 cdno. principal dos (2).* fis. 338 a 348 lb.$ fi. 373 cdno. principal tres (3).540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 6algunas pruebasf luego de lo cual se concedio a las partes un terminopara que se formulasen los correspondientes alegatos de conclusion.oportunamente la opositora estimo que fueron factoreseconomicos los que llevaron al reclamante a salir de","en torno al despojo alegado estimo$ fls. 7 a 8 cdno. del tribunal.7 fl 56. ib. o8 fls. 59 a 63 ib.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 7haberse sucedido por cuanto la enajenacion del predio por parte delsolicitante se llevo a cabo dentro de un contexto de violencia. consideroasi que deberia concederse la implorada proteccion del derechofundamental a la restitucion?.el minist","c-715 de 2012. magistrado ponente: dr. luis ernestovargas silva.13 fls. 32 a 41 cdno. 1 principal.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 8abandonadas forzosamente, en calidad de propietario y compañerapermanente, respectivamente, al momento de los hechos victimizantes,del predio rural denominado parcela n* 12 santa monica, ubicado en lavereda la javilla, corregimiento de aguac","y 43 ib.15 fls. 54 a 56 ib.16 fls. 57 y 58 ib.17 fis. 99 a 104 ib.18 articulo 78 ley 1448 de 2011.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 9establecido entonces este vinculo de manuel mariarodriguez florian y flor merinda diaz lerma con la heredadobjeto de la solicitud de restitucion como asi mismo su condicion devictimas de desplazamiento, cuanto incumbe ahora es detenerse en e","frente carlos velasco villamizar, en cucuta(http:/awww.acnur.org/i3/uploads/pics/2182.pdf?view=1).540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 10villamizar, se tienen informes periodisticos que dan cuenta de lasituacion de orden publico que abarcan ese periodo de tiempo narradopor los solicitantes??.igualmente, varios de los testigos citados dieron a conocerque en la zona de ubicaci","0.33.38.25 [b, record; 00.52.03.26 fl 5264. ib. record: 00.10.15.27 fl. 524. lb. record: 00.18.36.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 41sin embargo, la demostracion de esos puntales no resultabastante para lograr el exito de la especifica proteccion por la que aquise propende. pues no cabe perder de mira que en estos escenarios, esmenester ademas llegar a la clara persuasio"," contiendas, no basta la palmariacomprobacion de esa calidad de victima como tampoco con acreditar540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 12diamantinamente sucesos de violencia en la zona que puedan serligados al conflicto armado; ni siquiera si a la par se comprueba que elbien fue dejado al desgaire de algun modo (abandonado, vendido, etc.)cuanto que, por sobremanera, verifica","olombiano (...) (sentencia c-781de 10 de octubre de 2012. m.p. dra. maria victoria calle correa).540013121002201500389 01 ,solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 13favor de jose calderon, por imposicion del mismo jefe guerrillero,habiendo recibido por ese convenio, solo la suma de $1.300.000.00.tal fue en efecto cuanto este dijo dentro del marco de laactuacion administrativa ante la unidad de restituc","oy con mi hijo.29 fl 64 cano. 1 principal.30 fl. 557a. cdno. principal tres (3). record: 00.14.10.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 1 4entonces el dijo: espere un momentico; vamos a ver que hacemos mientrasyo puedo vender la parcela o la mejora a ver para que nos vayamos y si, pusoen venta la mejora. y yo tenga a mi hijo ahi, aguantelo; a no dejarlo salir, ahilo detuvimos","a que esta proceda a relevarla de la carga de la prueba (sentenciac-253a/12 corte constitucional).540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 45devenir de hechos poco menos perceptibles que las mas de las vecesocurren de manera privada y/o velada haciendolos casi queinapreciables a personas distintas de los que debieron padecerlos, porlo que, en casos tales, resulta hasta justifica","a en asunto que por suevidente conexidad con lo que asi se discute, tiene plena aplicacion, *(...)540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 16no porque se trate de un proceso de justicia transicional, el juzgador estarelevado de verificar el contenido y efecto de la prueba en su conjunto, pues,de ninguna manera la existencia de presunciones legales implica asumir comocierto lo qu","os fraudulentamente, iniciado por edgardo manuelbarros redondo).35 fls, 75 a 98 cano. 1 principal.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 17en adelante (memorese que los hechos victimizantes aqui relatadosdatan de 1996), ni por asomo hacen mencion de situacion alguna relativacon reclutamiento forzado propiciado por las guerrillas de izquierda.lo que tampoco se descubre de otras",".58.40 b. record: 00.13.04.41 ib. record: 00.19.30.42 [b. record: 00.19.45.43 [b, record: 00.20.20.540013121002201500389 01 solichud de restitucion y formalizacion de tierras de manuel maria rodriguez floriian y otra. 18escuela), como nosotros todos (...)*** al tiempo mismo en que lo hacian suspropios hijos*, dejo en claro, frente al aducido reclutamiento, que (...)no su señoria, nunca; nosotros tuvimos pues los hijos que fue la niña en esetiempo olga joh","1.50 [b. record: 00.53.13.$1 [b. record: 00.53.33.52 lb. record: 00.54.06.53 [b. record: 00.57.38.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 19ademas de que provienen de vecinos parceleros que residen hace casidos decadas en la misma exacta zona y que por esa cercania seencuentran en situacion de privilegio para conocer de primera mano lascircunstancias de las que hablaron, refiri","24.55 b. record: 00.20.14.58 [b. record: 00.13.33.57 [b, record: 00.47.18.58 lb, record: 00.47.22.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 20conflicto armado (por ejemplo por ese hecho) cuanto importaverdaderamente es verificar si esa condicion provoco que se perdiera elderecho de propiedad sobre el predio. no por nada este diligenciamientoapunta derechamente a la restitucion de","el que va a quedar ahi. ya yo entendi que el iba a59 lb. record: 00.14.10.$0 [b, record: 00.14.18.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 94quedar ahi, que a mi lo que me tocaba era que la parcelita me la iban a quitar.vuelvo a donde el señor jose calderon y le digo que: entonces que es loque esta pasando? que el es el que me va a comprar eso y no tiene plata y eldice que no; e","7.$2 lb. record: 00.23.59.$3 lb. record: 00.23.20.64 [b, record: 00.24.01.$5 [b, record: 00.15.48.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 22tienda porque eso no me alcanzo para montar una tiendita, lo que llaman unchuzito ahi; pero no me fue bien porque eso no conocia bien la ciudad; no, lopuse en un mal sitio y practicamente eso (...)8,de lo que si hablo fue de jose calderon d","o,66 (b. record: 00.26.29.67 lb. record: 00.22.40.$8 [b. record: 00.32.53.$9 lb. record: 00.27.06.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 23cual dijere manuel maria, fue a quien se responsabilizo de provocarel gravisimo temor de que (...) ellos me ban a asesinar (...)70.sumado todo a que, como antes se dijo, salvo la tangencialmencion que hiciere zoraida sanchez ramirez en punt","5 de marzo de 2018, radicacionexpediente n* 540013121002201500252 01).73 fl. 65 cdno. 1 principal.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 24como fuere, con todo y que se tuviere por demostrado todoello, ni aun asi termina acreditandose con suficiencia que esos hechosfueron de veras los que constituyeron causa eficiente de la venta. a lomenos no si al propio tiempo se traen a cu","ion, cuyo monto comenzara a cobrarse a partir deltercer año (...) (fl. 54 vto. cdno. 1 principal).540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 25que en cualquier caso, desde la fecha de la notificacion de laadjudicacion al aqui solicitante (abril de 1993) y hasta cuando dijo quese produjo su desplazamiento y ya luego la venta del bien (a mediadosde 1996), habian transcurrido ya mas ","ciones contraidas por el enajenante (...).81 fl, 557a. cdno. principal tres (3). record: 00.24.26.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 26parcela, la mejora, por nueve millones quinientos mil (...)82, por supuestoque tan elevado precio, siguiendo muy de cerca su propio dicho, no sejustificaria desde que no concordaria con lo que enuncio de que deberiarecibir por la venta unic","escasos recursos, y estaransujetas en un todo a lo dispuesto por el capitulo x de la presente ley.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 27incluso se advirtio en la misma resolucion al señalar que en el eventode enajenacion o cesion del fundo, (...) el adquirente o cesionario deberareunir las mismas condiciones del beneficiario inicial (...).digase ahora que esa autorizacion q","ecidimos vender el predio para podemos ir, y evitar la muerte mia (...) (fl. 64cdno. 1 principal).540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 28condicion de victimas en el año de 2002, por hechos sucedidos en elmunicipio antioqueño de yondo (casabe) y comunicados enbarrancabermeja, diciendose que (...) cada rato se ve en elpueblo que la gente dice que viene la guerrilla que vienenl","ial para la atencion y reparacion integral a90 fl 74 cano. del tribunal.9 fl 65 cdno. 1 principal.540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 29las victimas, para que, si es del caso, provea los mecanismos de ayudaque resulten pertinentes para lograr la debida atencion requieran.finalmente, y en tanto que no aparece que se hubierencausado costas en las especificas condiciones señal"," orden de suspension deprocesos judiciales, administrativos o de cualquier otra naturaleza que540013121002201500389 01 >>a]solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 30se adelanten ante autoridades publicas o notariales en los cuales sehallaren comprometidos derechos sobre el indicado predio. oficiese.quinto.- conminase al director de la unidad para laatencion y reparacion integral a las victimas a que, p","uceso victimizante yla venta, que es presupuesto sine quanon para que tenga exito lapretension restitutoria, no se ofrece solucion distinta que la de negar lareclamada peticion. asi, entonces, habra de resolverse el asunto sin quesea menester ocuparse de las alegaciones de los opositores si del modoantes referido, y por pura sustraccion de materia, quedo suficientementesolucionado el conflicto.sin perjuicio de cuanto viene dicho, y dando cuenta que lo","o de aguaclara, municipio de cucuta,norte de santander.cuanto refiere con la relacion juridica de los solicitantesrespecto del fundo que aqui se pide restituir y para la epoca del despojoo del abandono, basta con decir que de acuerdo con la informacioninscrita en el folio de matricula inmobiliaria n* 260-151578**, el dichobien fue en comienzo adjudicado por el incora a la solicitante medianteresolucion n* 00390 de 7 abril de 199375 (anotacion n* 1); domin","ivil de decisionespecializada en restitucion de tierras, administrando justicia ennombre de la republica y por autoridad de la ley,resuelve:primero.- nieganse las peticiones formuladas pormanuel maria rodriguez florian y flor merinda diazlerma, de conformidad con las motivaciones que anteceden.segundo.- por consecuencia, excluyase del registro detierras despojadas y abandonadas forzadamente, la inscripcion queotrora se hiciera a favor de los señalados so"],"list_ubicacion":["uso enterar de la iniciacion de laaccion al alcalde, personero y al comite de justicia transicional delmunicipio de cucuta, al igual que a su par del departamento de norte desantander?.surtida la notificacion de blanca cecilia carreñocardenas), procedio a traves de apoderado judicial a pronunciarsefrente a los hechos de la solicitud de restitucion de tierras, indicandobasicamente que el citado comandante juan carlos del grupoguerrillero eln al cual se atribu","l eln ha sido preponderante por largo tiempo y hasta finales de los noventa sobrelas otras organizaciones alzadas en armas que tienen presencia en el departamento. los frentes deleln ubicados en norte de santander hacen parte del frente de guerra nororiental, la estructura belicacon presencia historica en el departamento y la mas activa del eln, cuya expansion se ha dado alrededorde la explotacion del oleoducto caño limon-coveñas. esta organizacion tiene pre","to. los frentes deleln ubicados en norte de santander hacen parte del frente de guerra nororiental, la estructura belicacon presencia historica en el departamento y la mas activa del eln, cuya expansion se ha dado alrededorde la explotacion del oleoducto caño limon-coveñas. esta organizacion tiene presencia a traves de losfrentes efrain pabon pabon y juan fernando porras en el centro y sur del departamento, el carlosarmando cacua guerrero en la zona del cata","eoducto caño limon-coveñas. esta organizacion tiene presencia a traves de losfrentes efrain pabon pabon y juan fernando porras en el centro y sur del departamento, el carlosarmando cacua guerrero en la zona del catatumbo; y el frente carlos velasco villamizar, en cucuta(http:/awww.acnur.org/i3/uploads/pics/2182.pdf?view=1).540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 10villamizar, se t","on el folio de matriculainmobiliaria n* 260-151578 y numero predial 00-01-0002-0216-00,ubicado en la vereda la javilla del corregimiento aguaclara delmunicipio de cucuta (norte de santander). igualmente, peticionaron quese impartiesen las demas ordenes previstas en el articulo 91 de la citadaley 1448.los pedimentos asi sintetizados encontraron soporte en loshechos que, seguidamente, asi se compendian y relacionan:manuel. maria rodriguez florian adquirio e"," y del bancoagrario de colombia. a su vez, dispuso enterar de la iniciacion de laaccion al alcalde, personero y al comite de justicia transicional delmunicipio de cucuta, al igual que a su par del departamento de norte desantander?.surtida la notificacion de blanca cecilia carreñocardenas), procedio a traves de apoderado judicial a pronunciarsefrente a los hechos de la solicitud de restitucion de tierras, indicandobasicamente que el citado comandante juan","refirio que el documento aportado como prueba dedesplazamiento sufrido por los reclamantes da cuenta de su ocurrenciaen el mes de marzo de 2002 en el municipio de yondo -antioquia-declarado el 2 de abril de 2002 en el municipio de barrancabermeja. deotro lado, estimo no ser viable considerar despojo la decision contendidaen un acto administrativo por medio del cual el otrora incora declarola caducidad administrativa de la resolucion de adjudicacion hecha ","do por los reclamantes da cuenta de su ocurrenciaen el mes de marzo de 2002 en el municipio de yondo -antioquia-declarado el 2 de abril de 2002 en el municipio de barrancabermeja. deotro lado, estimo no ser viable considerar despojo la decision contendidaen un acto administrativo por medio del cual el otrora incora declarola caducidad administrativa de la resolucion de adjudicacion hecha a losreclamantes mediante resolucion n* 0390 de 7 de abril de 1993, ","al. finalmente indico quede acuerdo con registro unico de victimas, se tiene conocimiento dehechos victimizantes contra la reclamante ocurridos en el municipio deyondo -antioquia- el 25 de marzo de 2002 sin que se hubieredenunciado desplazamiento anterior en el corregimiento de aguaclaradel municipio de cucuta.por su parte, la unidad de restitucion de tierras, enrepresentacion de los solicitantes, adujo que estaba acreditada lacalidad de victima de manuel","s en el municipio deyondo -antioquia- el 25 de marzo de 2002 sin que se hubieredenunciado desplazamiento anterior en el corregimiento de aguaclaradel municipio de cucuta.por su parte, la unidad de restitucion de tierras, enrepresentacion de los solicitantes, adujo que estaba acreditada lacalidad de victima de manuel maria rodriguez florian asicomo el elemento de temporalidad, por cuanto se vio obligado aabandonar su predio en el año 1996 por temor a ser a","mento de los hechos victimizantes,del predio rural denominado parcela n* 12 santa monica, ubicado en lavereda la javilla, corregimiento de aguaclara, municipio de cucuta,norte de santander.cuanto refiere con la relacion juridica de los solicitantesrespecto del fundo que aqui se pide restituir y para la epoca del despojoo del abandono, basta con decir que de acuerdo con la informacioninscrita en el folio de matricula inmobiliaria n* 260-151578**, el dichob","el conflicto armado.asi se comprueba, por ejemplo, con la informacioncontenida y recolectada en el documento de analisis del contexto deviolencia del municipio del area rural de cucuta?? aportado por la unidadde restitucion de tierras, en el cual, entre otros aspectos, se relieva quedurante la primera mitad de la decada del 70 y hasta la primera mitad dela decada de los años noventa, el eln es el unico grupo guerrillero condominio en norte de santander. t","restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 28condicion de victimas en el año de 2002, por hechos sucedidos en elmunicipio antioqueño de yondo (casabe) y comunicados enbarrancabermeja, diciendose que (...) cada rato se ve en elpueblo que la gente dice que viene la guerrilla que vienenlos paracos y le toca a uno salir corriendo entoncesdebido a eso decidi venirme (...)'9. desde luego que no podriaacusarse que esa omisio"," los señalados solicitantes respecto del prediodenominado parcela n* 12 santa monica, ubicado en la vereda lajavilla, del corregimiento aguaclara del municipio de cucuta (norte desantander), al que le corresponde el folio de matricula inmobiliaria n*260-151578 y el numero predial 00-01-0002-0216-00, y que apareceidentificado y descrito en la solicitud como sus anexos. oficiese.tercero.- (cancelense las medidas  cautelaresordenadas por cuenta de este asunt","a unarea de 15 hectareas y 4618 m?, distinguido con el folio de matriculainmobiliaria n* 260-151578 y numero predial 00-01-0002-0216-00,ubicado en la vereda la javilla del corregimiento aguaclara delmunicipio de cucuta (norte de santander). igualmente, peticionaron quese impartiesen las demas ordenes previstas en el articulo 91 de la citadaley 1448.los pedimentos asi sintetizados encontraron soporte en loshechos que, seguidamente, asi se compendian y r","o y compañerapermanente, respectivamente, al momento de los hechos victimizantes,del predio rural denominado parcela n* 12 santa monica, ubicado en lavereda la javilla, corregimiento de aguaclara, municipio de cucuta,norte de santander.cuanto refiere con la relacion juridica de los solicitantesrespecto del fundo que aqui se pide restituir y para la epoca del despojoo del abandono, basta con decir que de acuerdo con la informacioninscrita en el folio de","de algo parecido. asi lomanifesto por ejemplo zoraida sanchez ramirez, quien luego deadmitir que en el sector funcionaba (...) la escuelita ahi de la vereda quela hicimos nosotros (...)'38 y de afirmar que alli tuvo sus hijos estudiando,cuando derechamente fue preguntada acerca de reclutamientos quehiciere el eln en ese escuela, manifesto de manera rotunda que (...)no, mis hijos pues estudiaron ahi, siempre estaba pendiente de mis hijos (...)37y que a ","enamientos por cuenta de dichogrupo para que hiciere parte de sus filas*. otro tanto mencionoaldemar zuleta giraldo, quien amen de reafirmar que en lavereda funcionaba la escuela*? e incluso que tambien sabia que el hijodel aqui solicitante se la pasaba (...) estudiando y trabajando en la parcelatambien cuando no tenian estudio; igual que todos nosotros (...)*?, en puntode su conocimiento sobre intimidaciones del eln para forzar alreclutamiento de los ","no habian otras personas que tenian quevender; no se que pasaria tiempo despues??.estado de cosas que no permite sino resumir el asunto enque, en esa vereda y por esa epoca, y respecto de una generalidad deparceleros con situaciones de veras muy semejantes a las de lossolicitantes, solo a estos intentaron reclutarle un hijo y, asimismo, no masque ellos fueron obligados a vender el terreno. algo extraño por decir lomenos.70 fl 64 cdno. 1 principal.71 fl","te, la inscripcion queotrora se hiciera a favor de los señalados solicitantes respecto del prediodenominado parcela n* 12 santa monica, ubicado en la vereda lajavilla, del corregimiento aguaclara del municipio de cucuta (norte desantander), al que le corresponde el folio de matricula inmobiliaria n*260-151578 y el numero predial 00-01-0002-0216-00, y que apareceidentificado y descrito en la solicitud como sus anexos. oficiese.tercero.- (cancelense las ","undamento en la ley 1448 de 2011, solicitaron se les protegieresu derecho fundamental a la restitucion y formalizacion de tierras,respecto del predio rural denominado parcela n? 12, el cual presenta unarea de 15 hectareas y 4618 m?, distinguido con el folio de matriculainmobiliaria n* 260-151578 y numero predial 00-01-0002-0216-00,ubicado en la vereda la javilla del corregimiento aguaclara delmunicipio de cucuta (norte de santander). igualmente, petic"," otra. 8abandonadas forzosamente, en calidad de propietario y compañerapermanente, respectivamente, al momento de los hechos victimizantes,del predio rural denominado parcela n* 12 santa monica, ubicado en lavereda la javilla, corregimiento de aguaclara, municipio de cucuta,norte de santander.cuanto refiere con la relacion juridica de los solicitantesrespecto del fundo que aqui se pide restituir y para la epoca del despojoo del abandono, basta con dec",".asi se comprueba, por ejemplo, con la informacioncontenida y recolectada en el documento de analisis del contexto deviolencia del municipio del area rural de cucuta?? aportado por la unidadde restitucion de tierras, en el cual, entre otros aspectos, se relieva quedurante la primera mitad de la decada del 70 y hasta la primera mitad dela decada de los años noventa, el eln es el unico grupo guerrillero condominio en norte de santander. tambien la infor","es con desapariciones forzadas, asesinatos selectivos,masacres y secuestros perpetrados por el eln, en la zona de cucuta,tanto en la zona urbana como rural (incluyendo el corregimiento deaguaclara), acaecidos entre los años de 1995 y 20012 como por iguallo dice el documento diagnostico departamental de norte desantander?. incluso, de la misma zona de aguaclara, la javilla y puerto19 fis. 75 a 98 cano. 1 principal.20 fls. 415 a 436 cdno. principal tres","o en cuanto hace con las razones del despojo.tal sucede, primeramente, al parar mientes en lo que señalael documento de analisis de contexto del area rural de cucuta que fuerarecaudado durante la etapa administrativa en cuya semblanza historica,que recoge un espacio de tiempo que abarca incluso desde la decadade los años 70 a la actualidad?', se limita a enunciar -de manera genericapor demas- la presencia de diversos actores armados legales (epl,farc ","informes2010fierra_conflicto/despojo_tierras_baja.paf. p. 71.) y por otra, que esos datos reseñados en el documento de analisis de contexto(dac) area rural de cucuta -punto 2.1.1. desplazamiento forzado y despojo de tierras por partede este accionar insurgente- (fis. 80 a 81 vto. cdno. 1 principal), en el que se hizo mencion de otrosdos sucesos que son los que mas se acomodan a ese invocado patron y que se corresponden con lasdeclaraciones de reclaman","izacion de tierras,respecto del predio rural denominado parcela n? 12, el cual presenta unarea de 15 hectareas y 4618 m?, distinguido con el folio de matriculainmobiliaria n* 260-151578 y numero predial 00-01-0002-0216-00,ubicado en la vereda la javilla del corregimiento aguaclara delmunicipio de cucuta (norte de santander). igualmente, peticionaron quese impartiesen las demas ordenes previstas en el articulo 91 de la citadaley 1448.los pedimentos asi sin","o de 2002 adjudico nuevamente el inmueble, en estaocasion, a favor de blanca cecilia carreño cardenas, quienactualmente y segun consta en el folio de matricula inmobiliaria n* 260-151578, ostenta su titularidad.del tramite ante el juzgado:540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 4el juzgado segundo civil del circuito especializado enrestitucion de tierras de cucuta admitio la ac","r parte del juez deconocimiento, limito su intervencion a oponerse a la solicitud decancelacion del gravamen que recae sobre el predio distinguido conmatricula inmobiliaria n* 260-151578 constituido mediante escriturapublica n* 665 de 9 de abril de 2007 otorgada en la notaria sexta decucuta y solicito el pago de la compensacion que contempla el articulo98 de la ley 1448 de 2011, a la cual dijo tener derecho por haber actuadocon buena fe exenta de culpa al"," fundo que aqui se pide restituir y para la epoca del despojoo del abandono, basta con decir que de acuerdo con la informacioninscrita en el folio de matricula inmobiliaria n* 260-151578**, el dichobien fue en comienzo adjudicado por el incora a la solicitante medianteresolucion n* 00390 de 7 abril de 199375 (anotacion n* 1); dominio quetuvo hasta cuando la misma entidad emitio la resolucion n* 007 de 28de enero de 2002 por la que decreto la caducidad adm","ta monica, ubicado en la vereda lajavilla, del corregimiento aguaclara del municipio de cucuta (norte desantander), al que le corresponde el folio de matricula inmobiliaria n*260-151578 y el numero predial 00-01-0002-0216-00, y que apareceidentificado y descrito en la solicitud como sus anexos. oficiese.tercero.- (cancelense las medidas  cautelaresordenadas por cuenta de este asunto, incluyendo la inscripcion de lademanda y la orden de prohibicion de enaj","ste asunto, incluyendo la inscripcion de lademanda y la orden de prohibicion de enajenar, que pesan sobre el bieninmueble distinguido con el folio de matricula inmobiliaria n* 260-151578de la oficina de registro de instrumentos publicos de cucuta. oficiese.cuarto.- cancelese por igual la orden de suspension deprocesos judiciales, administrativos o de cualquier otra naturaleza que540013121002201500389 01 >>a]solicitud de restitucion y formalizacion de tier","e tierras,respecto del predio rural denominado parcela n? 12, el cual presenta unarea de 15 hectareas y 4618 m?, distinguido con el folio de matriculainmobiliaria n* 260-151578 y numero predial 00-01-0002-0216-00,ubicado en la vereda la javilla del corregimiento aguaclara delmunicipio de cucuta (norte de santander). igualmente, peticionaron quese impartiesen las demas ordenes previstas en el articulo 91 de la citadaley 1448.los pedimentos asi sintetizados en","adjudico nuevamente el inmueble, en estaocasion, a favor de blanca cecilia carreño cardenas, quienactualmente y segun consta en el folio de matricula inmobiliaria n* 260-151578, ostenta su titularidad.del tramite ante el juzgado:540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 4el juzgado segundo civil del circuito especializado enrestitucion de tierras de cucuta admitio la accion ordenand","l juez deconocimiento, limito su intervencion a oponerse a la solicitud decancelacion del gravamen que recae sobre el predio distinguido conmatricula inmobiliaria n* 260-151578 constituido mediante escriturapublica n* 665 de 9 de abril de 2007 otorgada en la notaria sexta decucuta y solicito el pago de la compensacion que contempla el articulo98 de la ley 1448 de 2011, a la cual dijo tener derecho por haber actuadocon buena fe exenta de culpa al haber realiz"," aqui se pide restituir y para la epoca del despojoo del abandono, basta con decir que de acuerdo con la informacioninscrita en el folio de matricula inmobiliaria n* 260-151578**, el dichobien fue en comienzo adjudicado por el incora a la solicitante medianteresolucion n* 00390 de 7 abril de 199375 (anotacion n* 1); dominio quetuvo hasta cuando la misma entidad emitio la resolucion n* 007 de 28de enero de 2002 por la que decreto la caducidad administrativa d"," ubicado en la vereda lajavilla, del corregimiento aguaclara del municipio de cucuta (norte desantander), al que le corresponde el folio de matricula inmobiliaria n*260-151578 y el numero predial 00-01-0002-0216-00, y que apareceidentificado y descrito en la solicitud como sus anexos. oficiese.tercero.- (cancelense las medidas  cautelaresordenadas por cuenta de este asunto, incluyendo la inscripcion de lademanda y la orden de prohibicion de enajenar, que pes",", incluyendo la inscripcion de lademanda y la orden de prohibicion de enajenar, que pesan sobre el bieninmueble distinguido con el folio de matricula inmobiliaria n* 260-151578de la oficina de registro de instrumentos publicos de cucuta. oficiese.cuarto.- cancelese por igual la orden de suspension deprocesos judiciales, administrativos o de cualquier otra naturaleza que540013121002201500389 01 >>a]solicitud de restitucion y formalizacion de tierras de manuel"]}

In [4]:
df_victimario_test = pd.DataFrame(raw_text["list_victimario"], columns =['Sentencia'])
df_perdida_test = pd.DataFrame(raw_text["list_perdida"], columns =['Sentencia'])
df_solicitante_test = pd.DataFrame(raw_text["list_solicitante"], columns =['Sentencia'])
df_decision_test = pd.DataFrame(raw_text["list_decision"], columns =['Sentencia'])
df_ubicacion_test = pd.DataFrame(raw_text["list_ubicacion"], columns =['Sentencia'])

In [5]:
# Working with the data of decision #
# df_decision = pd.read_csv("resources/list decision.csv")

df_decision_test["Sentencia_cleaned"] = df_decision_test["Sentencia"].apply(lambda x: process_paragraph(x))
df_decision_test.head()

,Sentencia,Sentencia_cleaned
0,,
1,veinte de marzo de dos mil dieciocho.radicacio...,veint marz dos mil diecioch radicacion n 54001...
2,z lerma.discutido y aprobado por la sala en se...,z lem discut aprob sal sesion 21 febrer 2018 s...
3,iva especial de gestion derestitucion de tierr...,iba especial gestion destitucion tierr despoj ...
4,o solicitud de restitucion y540013121002201500...,o solicitud restitucion 540013121002201500389 ...


In [6]:
import pickle
path_model ="/Users/dmurcia/Documents/personal/DS4A/Proyecto final/DS4A/Backend/outputs/models/"
path_transformer = "/Users/dmurcia/Documents/personal/DS4A/Proyecto final/DS4A/Backend/outputs/transformers/"
filename_model = "fitted_model_decision.pkl"
filename_transformer = "transformer_model_decision.pkl"
model = pickle.load(open(path_model+filename_model,'rb'))
tfidf_vectorizer = pickle.load(open(path_transformer+filename_transformer,'rb'))

In [34]:
# Preprocessing of text column #
test_input = df_decision_test["Sentencia"].apply(lambda x: process_paragraph(x))

In [35]:
# Vecotrizing column text using tfidf #
test_input = tfidf_vectorizer.fit_transform(test_input)

# usar predict proba # .targe value_counts() 0 y 1 - Output is a ndarray, [p0, p1] with the probability of each label
test_pred = model.predict_proba(test_input)

In [63]:
#asociar predicion
i = 0
json_output = {}
ind = np.argpartition(test_pred[:,1], -6)[-6:]
probability = test_pred[:,1][ind]
for value in ind:
    sentencia = df_decision_test["Sentencia"][value]
    print(f"probability:{probability[i]} , sentencia: {sentencia}")
    i = i + 1


probability:0.28516902146010187 , sentencia: z lerma.discutido y aprobado por la sala en sesion de 21 de febrerode 2018, segun acta n* 05 de la misma fecha.decidese la solicitud de restitucion y formalizacion detierras presentada por manuel maria rodriguez florian y flormerinda diaz lerma a cuya prosperidad se opone blancacecilia carreño cardenas.antecedentes:en ejercicio de la facultad otorgada por el articulo 82 de laley 1448 de 2011, la unidad administrativa especial de gestion derestitucion de t
probability:0.2902476098255847 , sentencia:  region en el año de 1996, abandonarforzosamente el predio y, en consecuencia, como unica solucion540013121002201500389 01 solicitud de restitucion y formalizacion de tierras de manuel maria rodriguez florian y otra. 3enajenarlo. la situacion narrada le impidio pagar el credito que, almomento de adquirir el predio, habia asumido por el termino de quinceaños.posteriormente, el citado comandante juan carlos, le diola orden al reclamante que negociar